<a href="https://colab.research.google.com/github/NajwaMoursli/HPCA_Project/blob/main/GPU_Najwachille.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HPCA Project : Batch Merge and Merge Sort Path

In [ ]:
# Preparation
!nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
!mkdir -p src/repos src/obj
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-0cg9ph_h
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-0cg9ph_h
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=a592ddbc63cd9e7835d29133a1ccf8a177bc5dccef86addd29fe6ce9e3a61844
  Stored in directory: /tmp/pip-ephem-wheel-cache-jgwjhqn2/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [ ]:
# Type de carte graphique. Mieux avec Tesla T4 ou T plus élevé
!nvidia-smi

Tue Dec 15 14:40:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!cd src  && rm -Rf Makefile && wget -P . https://gist.github.com/NajwaMoursli/f1b95b7d8ceb8d1abb8c446ece66960e/raw/Makefile

--2020-12-15 14:40:19--  https://gist.github.com/NajwaMoursli/f1b95b7d8ceb8d1abb8c446ece66960e/raw/Makefile
Resolving gist.github.com (gist.github.com)... 13.114.40.48
Connecting to gist.github.com (gist.github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/NajwaMoursli/f1b95b7d8ceb8d1abb8c446ece66960e/raw/Makefile [following]
--2020-12-15 14:40:20--  https://gist.githubusercontent.com/NajwaMoursli/f1b95b7d8ceb8d1abb8c446ece66960e/raw/Makefile
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 712 [text/plain]
Saving to: ‘./Makefile’

Makefile            100%[===================>]     712  --.-KB/s    in 0s      

2020-12-15 14:40:20 (33.8 MB/s) - ‘./Makefile’ saved 

In [ ]:
!rm -Rf src/repos/functions
!git clone https://gist.github.com/NajwaMoursli/5fe67ed5c22e84ba7a6331fcf4c79249 src/repos/functions

Cloning into 'src/repos/functions'...
remote: Enumerating objects: 12, done.
remote: Total 12 (delta 0), reused 0 (delta 0), pack-reused 12
Unpacking objects: 100% (12/12), done.


In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2200.000
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4400.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [ ]:
!hostname

579a74ff6ecf


# Question 1.1 : Algo Merge Path of the Project

In [ ]:
%%%%cuda --name main.cu
/* To parallelize the algorithm, the grid has to be extended to the maximum size
equalt to max(|A|,|B|)x max(|A|,|B|). We note K_0 and P_0 respectively the
low point and the high point of the ascending diagonals Delta_k.
On GPU, each thread k in [[0,|A|+|B|-1]] is responsible of ONE diagonal.
It finds the intersection of the merge path and the diagonal Delta_k with a
binary search described in Algorithm 2.
(1) For |A| + |B| <= 1024 write a kernel "mergeSmall_k" that merges A and B
using only one block of threads
*/

#include<stdio.h>
#include<math.h>
#include<algorithm>
#include<fstream> //to export data
#include<time.h> //random numbers
#include "repos/functions/functions.hpp"
#define testCUDA(error) (testCUDA(error,__FILE__,__LINE__))


//####################### KERNEL #################################
__global__ void mergeSmall_k(int *A, int dim_A, int *B, int dim_B, int *M){
  int idx = threadIdx.x;//thread index 
  int K_x, K_y, P_x, P_y, Q_x, Q_y;
  int off_set;
  if(idx < (dim_A + dim_B)){
    if(idx>dim_A){
      K_x = idx - dim_A; //(K_x, K_y): low point of diagonal
      K_y = dim_A;
      P_x = dim_A; //(P_x,P_y): high point of diagonal
      P_y = idx - dim_A;
    }else{
      K_x = 0;
      K_y = idx;
      P_x = idx;
      P_y = 0;
    }
    while(1){
      off_set = (abs(K_y-P_y)/2); //integer
      //distance on y == distance on x, because diagonal
      Q_x = K_x + off_set;
      Q_y = K_y - off_set;
      //offset is an "int" (integer), so it's rounded to the smaller integer -> Q will be closer to K
      if (Q_y>=0 && Q_x<=dim_B && (Q_y == dim_A || Q_x==0 || A[Q_y] > B[Q_x-1]) ){
        // (Q_y>=0 and Q_x<=B) -> Q is in the grid
        // Q_y=|A| -> Q is on the down border
        // Q_x=0   -> Q is on the left border
        // A[Q_y>B[Q_x-1] -> the "path" goes 'over Q' or 'pass through Q'
          if (Q_x == dim_B || Q_y == 0 || A[Q_y-1] <= B[Q_x]){
            // Q_x=|B| -> Q is on the right border
            // Q_y=0   -> Q in on the up border
            // A[Q_y-1]<=B[Q_x] -> the "path" goes 'under Q' or 'pass through Q'
              if (Q_y<dim_A && (Q_x == dim_B || A[Q_y]<= B[Q_x]) )
              //if Q is not on the down border (= if the "path" can go down) and it MUST go down
                M[idx] = A[Q_y]; //the "path" goes down
              else //if it can't go down
                M[idx] = B[Q_x]; //the "path" goes right (in B direction)

              break;
          }else{ //if the path is over Q but not under Q
              K_x = Q_x + 1; //move Q up by moving K up (updating Q_x to remain on diagonal)
              K_y = Q_y - 1;
          }
      }else{ //if the path is under Q
        P_x = Q_x - 1; //move Q down by moving P down (updating Q_x to remain on diagonal)
        P_y = Q_y + 1;
      }
    }
  }
}

//########################### WRAPPER ###################################
void wrapper_mergeSmall(int *A, int dim_A, int *B, int dim_B, int *M, float & duration){
  int *A_GPU, *B_GPU, *M_GPU;

  //Initializating the timer:
  float TimerV;
  cudaEvent_t start, stop;
  testCUDA(cudaEventCreate(&start));
  testCUDA(cudaEventCreate(&stop));
  testCUDA(cudaEventRecord(start,0));

  //Initialization on GPU:
  testCUDA(cudaMalloc(&A_GPU, dim_A*sizeof(int)));
  testCUDA(cudaMalloc(&B_GPU, dim_B*sizeof(int)));
  testCUDA(cudaMalloc(&M_GPU, (dim_A+dim_B)*sizeof(int)));

  testCUDA(cudaMemcpy(A_GPU, A, dim_A*sizeof(int), cudaMemcpyHostToDevice));
  testCUDA(cudaMemcpy(B_GPU, B, dim_B*sizeof(int), cudaMemcpyHostToDevice));

  //Launching the operation on the GPU
  mergeSmall_k<<<1,1024>>>(A_GPU, dim_A, B_GPU, dim_B, M_GPU);

  //Bringing back the results from GPU to CPU:
  testCUDA(cudaMemcpy(M, M_GPU, (dim_A+dim_B)*sizeof(int), cudaMemcpyDeviceToHost));

  //Stop the timer:
  testCUDA(cudaEventRecord(stop,0));
  testCUDA(cudaEventSynchronize(stop));
  testCUDA(cudaEventElapsedTime(&TimerV, start, stop));
  printf("Execution time: %f ms\n", TimerV);
  duration = TimerV;

  //Freeing the GPU MEMORY
  testCUDA(cudaFree(A_GPU));
  testCUDA(cudaFree(B_GPU));
  testCUDA(cudaFree(M_GPU));
  testCUDA(cudaEventDestroy(start));
  testCUDA(cudaEventDestroy(stop));
}

//############################# MAIN ###################################
int main(){
  //Initialization:
  int *A, *B, *M;
  int d[10];
  float duration[10];
  for(int i=0; i<10; ++i)
    d[i] = pow(2,i+1); //2, 4, 8, ..., 1024

  for(int k=0; k<10; k++){
    srand (time(NULL));
    int dim_A = rand() % (d[k]-1) + 1; //choosing randomly the dimension
    int dim_B = d[k] - dim_A;
    if(dim_B > dim_A) //necessary condition: |A| > |B|
      std::swap(dim_A,dim_B);
    A = (int*)malloc(dim_A*sizeof(int));
    B = (int*)malloc(dim_B*sizeof(int));
    generate_random(A,dim_A,0,1000); //random generation of the input arrays
    A[dim_A/2] = 1000;
    mergesort(A,0,dim_A-1);
    generate_random(B,dim_B,0,1000);
    B[dim_B/2] = -1;
    mergesort(B,0,dim_B-1);
    M = (int*)malloc((dim_A+dim_B)*sizeof(int));

    printf("%i) \nA : \n",k);
    print_array(A,dim_A);
    printf("\n");

    printf("B: \n");
    print_array(B,dim_B);
    printf("\n");

    //###### WRAPPER CALL ######
    wrapper_mergeSmall(A, dim_A, B, dim_B, M, duration[k]);

    printf("\n");
    printf("M: \n");
    print_array(M, dim_A+dim_B);
    printf("\n \n \n");

    free(A);
    free(B);
    free(M);
  }
}



'File written in /content/src/main.cu'

In [ ]:
!cd src && make 

nvcc -lineinfo main.cu -o main  
main.cu(23): warning: variable "P_x" was set but never used



In [ ]:
!cd src && make run

./main
0) 
A : 
dim: 1 
1000 

B: 
dim: 1 
-1 

Execution time: 0.264512 ms

M: 
dim: 2 
-1 1000 

 
 
1) 
A : 
dim: 2 
488 1000 

B: 
dim: 2 
-1 922 

Execution time: 0.149248 ms

M: 
dim: 4 
-1 488 922 1000 

 
 
2) 
A : 
dim: 5 
206 234 488 894 1000 

B: 
dim: 3 
-1 333 910 

Execution time: 0.155264 ms

M: 
dim: 8 
-1 206 234 333 488 894 910 1000 

 
 
3) 
A : 
dim: 11 
79 149 206 234 361 378 488 894 910 922 1000 

B: 
dim: 5 
-1 300 351 377 933 

Execution time: 0.135168 ms

M: 
dim: 16 
-1 79 149 206 234 300 351 361 377 378 488 894 910 922 933 1000 

 
 
4) 
A : 
dim: 19 
149 206 234 300 333 351 361 377 378 488 488 500 624 771 894 910 922 933 1000 

B: 
dim: 13 
-1 0 302 474 562 595 709 723 751 755 812 832 919 

Execution time: 0.131968 ms

M: 
dim: 32 
-1 0 149 206 234 300 302 333 351 361  ... 755 771 812 832 894 910 919 922 933 1000 

 
 
5) 
A : 
dim: 47 
0 79 149 153 206 234 296 300 302 321  ... 894 907 910 919 922 922 933 954 987 1000 

B: 
dim: 17 
-1 201 269 331 375 391 52

# Question 1.2 : Upgrade Algo 

In [ ]:
%%cuda --name main.cu
/* For any size |A| + |B| = d sufficiently smaller than the global memory, write
TWO kernels that merge A and B using various blocks.
The first kernel "pathBig_k" finds the merge path and
the second one "mergeBig_k" merges A and B. */

#include<stdio.h>
#include<math.h>
#include<cassert>
#include<time.h>
#include<fstream> //to export data
#include<algorithm>
#include "repos/functions/functions.hpp"
#define testCUDA(error) (testCUDA(error,__FILE__,__LINE__))

#define NSM 4096 // 4096 optimal for execution time
//56 = number of multiprocessors (SM) -> prop.multiProcessorCount
#define Z 32 //Number of thread per block

//#################### KERNEL - find MERGE PATH in the PARTITION POINTS ##############################
//X_path -> horizontal (>) <-> "B"-direction
//Y_path -> vertical (v) <-> "A"-direction
__global__ void MergePath_partition(int *A_gpu, int dim_A, int *B_gpu, int dim_B, int *X_gpu, int *Y_gpu){
//Divide the work among the processors: PARTITIONING
//"idx" is the partition index: it goes from 0 to #partitions (=#NSM)
  int idx = min(blockIdx.x * (dim_A+dim_B) / NSM, dim_A+dim_B);// dim(M)=[(dim_A+dim_B)/NSM, ....,(dim_A+dim_B)]
  int K_x, K_y, P_x, P_y, Q_x, Q_y;
  int off_set;

  //Cross diagonal binary search for MERGE PATH at the PARTITION POINTS on GLOBAL MEMORY
  //Writing the MERGE PATH at PARTITION POINTS on the GLOBAL MEMORY
    if(idx>dim_A){
      K_x = idx - dim_A; //(K_x, K_y): low point of diagonal
      K_y = dim_A;
      P_x = dim_A;      //(P_x,P_y): high point of diagonal
      P_y = idx - dim_A;
    }else{
      K_x = 0;
      K_y = idx;
      P_x = idx;
      P_y = 0;
    }
    while(1){
      off_set = (abs(K_y-P_y)/2);
      Q_x = K_x + off_set;
      Q_y = K_y - off_set;
      if (Q_y>=0 && Q_x<=dim_B && (Q_y == dim_A || Q_x==0 || A_gpu[Q_y] > B_gpu[Q_x-1]) ){
      // A[Q_y]>B[Q_x-1] -> the "path" goes 'over Q' or 'pass through Q'
          if (Q_x == dim_B || Q_y == 0 || A_gpu[Q_y-1] <= B_gpu[Q_x]){
          // A[Q_y-1]<=B[Q_x] -> the "path" goes 'under Q' or 'pass through Q' -> Q is on the "path"
                X_gpu[idx] = Q_x;
                Y_gpu[idx] = Q_y;
                break;
          }else{
              K_x = Q_x + 1;
              K_y = Q_y - 1;
          }
      }else{
        P_x = Q_x - 1;
        P_y = Q_y + 1;
      }
    }
  //The Merge Path terminates in the point (dim_B, dim_A):s
  if(blockIdx.x==0 && threadIdx.x==0){
    X_gpu[dim_A+dim_B] = dim_B;
    Y_gpu[dim_A+dim_B] = dim_A;
  }
}
// threadIdx.x => thread’s index within a block 
// block Idx.x => the current block’s index within the grid 

//######################### KERNEL - BIG PATH #################################
__global__ void pathBig_k(int *X_gpu, int *Y_gpu, const int *A_gpu, const int *B_gpu, int dim_A, int dim_B){
  //idx_start, idx_end: beginning of the partition, end of the partition
  int idx_start = min(blockIdx.x * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int idx_end = min((blockIdx.x+1) * (dim_A+dim_B) / NSM, dim_A + dim_B);
  //x_start, y_start: coordinates of the first element of the MergePath in the partition
  int x_start = X_gpu[idx_start];
  int y_start = Y_gpu[idx_start];

  //Place Z elements from each input array in the SHARED MEMORY for each partition 
  // Z = 32 threads in a block using the local shared memory of the SM
  __shared__ int A_sh[Z], B_sh[Z];
  int idx_sh = threadIdx.x;
  int K_x, K_y, P_x, P_y, Q_x, Q_y;
  int off_set;

//Cross diagonal binary search for MERGE PATH on SHARED MEMORY
//Writing the MERGE PATH on the GLOBAL MEMORY
 while(idx_start + idx_sh <= idx_end && idx_sh < Z){
   //Placing the input arrays in the SHARED MEMORY
  if(idx_sh + y_start < dim_A) //in order to have valid indexes for "A_sh" and "A_gpu"
    A_sh[idx_sh] = A_gpu[idx_sh + y_start];
  if(idx_sh + x_start< dim_B)
    B_sh[idx_sh] = B_gpu[idx_sh + x_start];

    if(idx_sh + y_start + 1 > dim_A){
      //since we know already the Merge Path at the starting point: (x_start, y_start)
      //we want to look for the Merge Path from the "next diagonal", that's why "+1"
      K_x = x_start + (idx_sh + y_start + 1 - dim_A);
      K_y = dim_A;
      } else{
        K_x = x_start;
        K_y = idx_sh + y_start + 1;
        }
    if(idx_sh + x_start + 1 > dim_B){
      P_x = dim_B;
      P_y = y_start + (idx_sh + x_start + 1 - dim_B);
      } else{
        P_x = idx_sh + x_start + 1;
        P_y = y_start;
      }

      while(1){
      off_set = (abs(K_y-P_y)/2);// divided into 2 pairs of values (n=abs(K_y-P_y)), and sort them with a kernel function using n/2 blocks
      Q_x = K_x + off_set;
      Q_y = K_y - off_set;
      if ((Q_y>=y_start && Q_y>=0) && (Q_x<=(Z+x_start)<=dim_B) && ((Q_y == (Z+y_start) || Q_y == dim_A) || (Q_x==x_start || Q_x==0) || A_sh[Q_y - y_start] > B_sh[Q_x -1 - x_start]) ){
      // A[Q_y>B[Q_x-1] -> the "path" goes 'over Q' or 'pass through Q'
          if ((Q_x == (Z+x_start) || Q_x == dim_B) || (Q_y == y_start || Q_y == 0) || A_sh[Q_y -1 - y_start] <= B_sh[Q_x - x_start]){
                X_gpu[idx_start + idx_sh +1] = Q_x;
                Y_gpu[idx_start + idx_sh +1] = Q_y;

                break;
          }else{
              K_x = Q_x + 1;
              K_y = Q_y - 1;
          }
      }else{
        P_x = Q_x - 1;
        P_y = Q_y + 1;
      }
    }

    //Updating the indexes
    x_start = X_gpu[idx_start + Z]; //x_start = x_end
    y_start = Y_gpu[idx_start + Z]; //y_start = y_end
    idx_start += Z;
  }
}

//######################### KERNEL to MERGE  ##################################
__global__ void mergeBig_k(int *X_gpu, int *Y_gpu, int *A_gpu, int *B_gpu, int *M_gpu, int dim_A, int dim_B){

  int idx_start = min(blockIdx.x * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int idx_end = min((blockIdx.x+1) * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int x_start = X_gpu[idx_start];
  int y_start = Y_gpu[idx_start];

  //Place Z ponits of the MERGE PATH in the SHARED MEMORY
  __shared__ int X_sh[Z], Y_sh[Z];

  int idx_sh = threadIdx.x;// threadIdx.x => thread’s index within a block 


 while(idx_start + idx_sh < idx_end && idx_sh < Z){
   //Placing the MERGE PATH in the SHARED MEMORY
    X_sh[idx_sh] = X_gpu[idx_start + idx_sh + 1];
    Y_sh[idx_sh] = Y_gpu[idx_start + idx_sh + 1];

    if(idx_sh == 0 && X_sh[0] > x_start) //If the path is moving right
      M_gpu[idx_start] = B_gpu[x_start];
    else if(idx_sh ==0) //if the path is moving down
      M_gpu[idx_start] = A_gpu[y_start];

    if( idx_sh>0 && X_sh[idx_sh]>X_sh[idx_sh-1]) //If the path is moving right
      M_gpu[idx_start + idx_sh] = B_gpu[X_sh[idx_sh -1]];
    else if(idx_sh > 0) //if the path is moving down
      M_gpu[idx_start + idx_sh] = A_gpu[Y_sh[idx_sh -1]];

    //Update the indexes
    x_start = X_sh[Z-1];
    y_start = Y_sh[Z-1];
    idx_start += Z;
  }
}
//########################### WRAPPER FIND PATH ###################################
//thread are executed within thiq group => wrap (contains 32  threads)
void wrapper_MergeSort(int *a, int dim_A, int *b, int dim_B, int *m){
  int *A_GPU, *B_GPU, *M_GPU, *X_GPU, *Y_GPU;

  int count;
  cudaDeviceProp prop;
  testCUDA(cudaGetDeviceCount(&count));
  testCUDA(cudaGetDeviceProperties(&prop, count-1));
  printf("\nGlobal memory size in octet (bytes): %ld \n", prop.totalGlobalMem); //ld: long double
  printf("Max dimension of the two input arrays: %ld \n", prop.totalGlobalMem/(sizeof(int)*8)); 
  printf("Shared memory size per block: %ld \n", prop.sharedMemPerBlock);
  printf("Max dimension of the int array in the shared memory: %ld \n", prop.sharedMemPerBlock/(sizeof(int)*8)); 
  printf("Number of multiprocessors: %i \n", prop.multiProcessorCount); //NSM
  printf("Maximum number of thread per block: %li \n", prop.maxThreadsPerBlock);

  //Initilizating the timer:
  float TimerV;
  cudaEvent_t start, stop;
  testCUDA(cudaEventCreate(&start));
  testCUDA(cudaEventCreate(&stop));
  testCUDA(cudaEventRecord(start,0));

  //Initialization on GPU:
  testCUDA(cudaMalloc(&A_GPU, dim_A*sizeof(int)));
  testCUDA(cudaMalloc(&B_GPU, dim_B*sizeof(int)));
  testCUDA(cudaMalloc(&X_GPU, (dim_A+dim_B+1)*sizeof(int)));
  testCUDA(cudaMalloc(&Y_GPU, (dim_A+dim_B+1)*sizeof(int)));
  testCUDA(cudaMalloc(&M_GPU, (dim_A+dim_B)*sizeof(int)));
  //Copying the values form one ProcUni to the other ProcUnit
  testCUDA(cudaMemcpy(A_GPU, a, dim_A*sizeof(int), cudaMemcpyHostToDevice));
  testCUDA(cudaMemcpy(B_GPU, b, dim_B*sizeof(int), cudaMemcpyHostToDevice));

  printf("Max number of blocks available: %li \n",prop.maxGridSize[0] );

  printf("Number of blocks used: %i \n", NSM);
  printf("Number of thread per block: %i \n", Z);


  //##### KERNEL CALLS ######
  MergePath_partition<<<NSM,1>>>(A_GPU, dim_A, B_GPU, dim_B, X_GPU, Y_GPU);

  pathBig_k<<<NSM,Z>>>(X_GPU, Y_GPU, A_GPU, B_GPU, dim_A, dim_B);

  mergeBig_k<<<NSM,Z>>>(X_GPU, Y_GPU, A_GPU, B_GPU, M_GPU, dim_A, dim_B);

  //Bringing back the results from GPU to CPU:
  testCUDA(cudaMemcpy(m, M_GPU, (dim_A+dim_B)*sizeof(int), cudaMemcpyDeviceToHost));

  //Stop the timer:
  testCUDA(cudaEventRecord(stop,0));
  testCUDA(cudaEventSynchronize(stop));
  testCUDA(cudaEventElapsedTime(&TimerV, start, stop));
  printf("Execution time of FIND PATH: %f ms\n \n", TimerV);

  //Freeing the GPU MEMORY
  testCUDA(cudaFree(A_GPU));
  testCUDA(cudaFree(B_GPU));
  testCUDA(cudaFree(M_GPU));
  testCUDA(cudaFree(X_GPU));
  testCUDA(cudaFree(Y_GPU));
  testCUDA(cudaEventDestroy(start));
  testCUDA(cudaEventDestroy(stop));
}


//############################# MAIN ###################################
int main(){
  //##### INITIALIZATION ######
  int *A, *B, *M;
  int d = 1000000;
  srand (time(NULL));
  int dim_A = rand() % d;
  int dim_B = d - dim_A;
  if(dim_B > dim_A)
    std::swap(dim_A,dim_B);
  A = (int*)malloc(dim_A*sizeof(int));
  B = (int*)malloc(dim_B*sizeof(int));

  generate_random(A,dim_A,0,1000);
  A[dim_A/2]=-1;//To check if it is sorted correctly
  mergesort(A,0,dim_A-1);
  generate_random(B,dim_B,0,1000);
  B[dim_B/2] = 1000; //To check if it is sorted correctly
  mergesort(B,0,dim_B-1);
  M = (int*)malloc((dim_A+dim_B)*sizeof(int));

  printf("A: \n");
  print_array(A,dim_A);
  printf("\n");

  printf("B: \n");
  print_array(B,dim_B);
  printf("\n");

//##### WRAPPER CALL #####
  wrapper_MergeSort(A, dim_A, B, dim_B, M);


  printf("M: \n");
  print_array(M, dim_A+dim_B);
  printf("\n \n");

  free(A);
  free(B);
  free(M);
}

'File written in /content/src/main.cu'

In [ ]:
!cd src && make 

nvcc -lineinfo main.cu -o main  
main.cu(26): warning: variable "P_x" was set but never used

main.cu(84): warning: variable "P_x" was set but never used

main.cu: In function ‘void wrapper_MergeSort(int*, int, int*, int, int*)’:
main.cu:190:77: warning: format ‘%li’ expects argument of type ‘long int’, but argument 2 has type ‘int’ [-Wformat=]
   printf("Maximum number of thread per block: %li \n", prop.maxThreadsPerBlock);
                                                      ~~~~~~~~~~~~~~~~~~~~~~~^
main.cu:209:71: warning: format ‘%li’ expects argument of type ‘long int’, but argument 2 has type ‘int’ [-Wformat=]
   printf("Max number of blocks available: %li \n",prop.maxGridSize[0] );
                                                  ~~~~~~~~~~~~~~~~~~~~~^


In [ ]:
!cd src && make run

./main
A: 
dim: 541814 
-1 0 0 0 0 0 0 0 0 0  ... 999 999 999 999 999 999 999 999 999 999 

B: 
dim: 458186 
0 0 0 0 0 0 0 0 0 0  ... 999 999 999 999 999 999 999 999 999 1000 


Global memory size in octet (bytes): 17071734784 
Max dimension of the two input arrays: 533491712 
Shared memory size per block: 49152 
Max dimension of the int array in the shared memory: 1536 
Number of multiprocessors: 56 
Maximum number of thread per block: 1024 
Max number of blocks available: 2147483647 
Number of blocks used: 4096 
Number of thread per block: 32 
Execution time of FIND PATH: 4.397280 ms
 
M: 
dim: 1000000 
-1 0 0 0 0 0 0 0 0 0  ... 999 999 999 999 999 999 999 999 999 1000 

 


# Representation of the merge path 1.2 


In [ ]:
%%cuda --name main.cu 
/*The only way to use the shared memory if we want to work with several blocks.
 Otherwise, we would index the shared thread with the global one 
- first  kernel (pathBig_k) built the merge path on the global memory 
- second kernel (mergeBig_k) based on the previous path built to do the merge
 in the shared memory. */


#include<stdio.h>
#include<math.h>
#include<cassert>
#include<time.h>
#include<algorithm>
#include"repos/functions/functions.hpp"
#define testCUDA(error) (testCUDA(error,__FILE__,__LINE__))

#define NSM 4096 //optimal for execution time -> graph
// 56 = number of multiprocessors -> prop.multiProcessorCount
#define Z 32

//#################### KERNEL - find MERGE PATH in the PARTITION POINTS ##############################
//X_path -> horizontal (>) <-> "B"-direction
//Y_path -> vertical (v) <-> "A"-direction
 __global__ void MergePath_partition(int *A_gpu, int dim_A, int *B_gpu, int dim_B, int *X_gpu, int *Y_gpu){
//Divide the work: PARTITIONING
//"idx" is the partition index: it goes from 0 to #partitions (=#SPs)
  int idx = min(blockIdx.x * (dim_A+dim_B) / NSM, dim_A+dim_B);
  int K_x, K_y, P_x, P_y, Q_x, Q_y;
  int off_set;

    if(idx>dim_A){
      K_x = idx - dim_A; //(K_x, K_y): low point of diagonal
      K_y = dim_A;
      P_x = dim_A;      //(P_x,P_y): high point of diagonal
      P_y = idx - dim_A;
    }else{
      K_x = 0;
      K_y = idx;
      P_x = idx;
      P_y = 0;
    }
    while(1){
      off_set = (abs(K_y-P_y)/2);// integer
      Q_x = K_x + off_set;
      Q_y = K_y - off_set;
      if (Q_y>=0 && Q_x<=dim_B && (Q_y == dim_A || Q_x==0 || A_gpu[Q_y] > B_gpu[Q_x-1]) ){
      // A[Q_y]>B[Q_x-1] -> the "path" goes 'over Q' or 'pass through Q'
          if (Q_x == dim_B || Q_y == 0 || A_gpu[Q_y-1] <= B_gpu[Q_x]){
          // A[Q_y-1]<=B[Q_x] -> the "path" goes 'under Q' or 'pass through Q' -> Q is on the "path"
                X_gpu[idx] = Q_x;
                Y_gpu[idx] = Q_y;
                break;
          }else{
              K_x = Q_x + 1;
              K_y = Q_y - 1;
          }
      }else{
        P_x = Q_x - 1;
        P_y = Q_y + 1;
      }
    }
  if(blockIdx.x==0 && threadIdx.x==0){
    X_gpu[dim_A+dim_B] = dim_B;
    Y_gpu[dim_A+dim_B] = dim_A;
  }
}

//######################### KERNEL - BIG PATH #################################
__global__ void pathBig_k(int *X_gpu, int *Y_gpu, const int *A_gpu, const int *B_gpu, int dim_A, int dim_B){
  int idx_start = min(blockIdx.x * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int idx_end = min((blockIdx.x+1) * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int x_start = X_gpu[idx_start];
  int y_start = Y_gpu[idx_start];

  __shared__ int X_sh[Z], Y_sh[Z];//shared memory
  __shared__ int A_sh[Z], B_sh[Z];//shared memory
  int idx_sh = threadIdx.x;
  int K_x, K_y, P_x, P_y, Q_x, Q_y;
  int off_set;

 while(idx_start + idx_sh <= idx_end){
  if(idx_sh + y_start < dim_A )
    A_sh[idx_sh] = A_gpu[idx_sh + y_start];
  if(idx_sh + x_start< dim_B)
    B_sh[idx_sh] = B_gpu[idx_sh + x_start];
     __syncthreads();
    if(idx_sh + y_start + 1 > dim_A){
      K_x = x_start + (idx_sh + y_start + 1 - dim_A);
      K_y = dim_A;
      } else{
        K_x = x_start;
        K_y = idx_sh + y_start + 1;
        }
    if(idx_sh + x_start + 1 > dim_B){
      P_x = dim_B;
      P_y = y_start + (idx_sh + x_start + 1 - dim_B);
      } else{
        P_x = idx_sh + x_start + 1;
        P_y = y_start;
      }

      while(1){
      off_set = (abs(K_y-P_y)/2);//integer
      Q_x = K_x + off_set;
      Q_y = K_y - off_set;
      if ((Q_y>=y_start && Q_y>=0) && (Q_x<=(Z+x_start)<=dim_B) && ((Q_y == (Z+y_start) || Q_y == dim_A) || (Q_x==x_start || Q_x==0) || A_sh[Q_y - y_start] > B_sh[Q_x -1 - x_start]) ){
          __syncthreads();
      // A[Q_y>B[Q_x-1] -> the "path" goes 'over Q' or 'pass through Q'
          if ((Q_x == (Z+x_start) || Q_x == dim_B) || (Q_y == y_start || Q_y == 0) || A_sh[Q_y -1 - y_start] <= B_sh[Q_x - x_start]){
              __syncthreads();
          // A[Q_y-1]<=B[Q_x] -> the "path" goes 'under Q' or 'pass through Q' -> Q is on the "path"
                  X_sh[idx_sh] = Q_x;
                  Y_sh[idx_sh] = Q_y;
                  //X_gpu[idx_start + idx_sh +1] = Q_x;
                  //Y_gpu[idx_start + idx_sh +1] = Q_y;
                 __syncthreads();
                break;
          }else{
              K_x = Q_x + 1;
              K_y = Q_y - 1;
          }
      }else{
        P_x = Q_x - 1;
        P_y = Q_y + 1;
      }
    }
    //x_start = X_gpu[idx_start + Z];
    //y_start = Y_gpu[idx_start + Z];
     x_start = X_sh[Z-1];
     y_start = Y_sh[Z-1];
     X_gpu[idx_start + idx_sh +1] = X_sh[idx_sh];
     Y_gpu[idx_start + idx_sh +1] = Y_sh[idx_sh];
      __syncthreads();
    idx_start += Z; //update index !!!
  }
}

//######################### KERNEL to MERGE  ##################################
__global__ void mergeBig_k(int *X_gpu, int *Y_gpu, int *A_gpu, int *B_gpu, int *M_gpu, int dim_A, int dim_B){

  int idx_start = min(blockIdx.x * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int idx_end = min((blockIdx.x+1) * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int x_start = X_gpu[idx_start];
  int y_start = Y_gpu[idx_start];

  __shared__ int X_sh[Z], Y_sh[Z];
  __shared__ int A_sh[Z], B_sh[Z];
  int idx_sh = threadIdx.x;

 while(idx_start + idx_sh < idx_end){ 
   if(idx_sh < Z-1 && idx_sh + y_start < dim_A){
     A_sh[idx_sh] = A_gpu[idx_sh + y_start];
   }
  if(idx_sh < Z-1 && idx_sh + x_start<= dim_B){
     B_sh[idx_sh] = B_gpu[idx_sh + x_start];
   }
    X_sh[idx_sh] = X_gpu[idx_start + idx_sh + 1];
    Y_sh[idx_sh] = Y_gpu[idx_start + idx_sh + 1];
    printf("X[%i] = %i, Y[%i] = %i", idx_sh, X_sh[idx_sh], idx_sh,Y_sh[idx_sh]);
    __syncthreads();
    if(idx_sh == 0 && X_sh[0] > x_start)
      M_gpu[idx_start] = B_gpu[x_start];
    else if(idx_sh ==0)
      M_gpu[idx_start] = A_gpu[y_start];

    if( idx_sh>0 && X_sh[idx_sh]>X_sh[idx_sh-1])
       // M_gpu[idx_start + idx_sh] = B_gpu[X_sh[idx_sh -1]];
        M_gpu[idx_start + idx_sh -1] = B_gpu[X_sh[idx_sh-1]];
        //M[idx_start + idx_sh] = B[idx_sh + x_start];
    else if(idx_sh > 0)
       M_gpu[idx_start + idx_sh] = A_gpu[Y_sh[idx_sh -1]];
       M_gpu[idx_start + idx_sh -1] = A_gpu[Y_sh[idx_sh-1]];

     __syncthreads();
    x_start = X_sh[Z-1];
    y_start = Y_sh[Z-1];
    idx_start += Z; //update index !!!
  }
}
//########################### WRAPPER FIND PATH ###################################
void wrapper_FindPartition(int *a, int dim_A, int *b, int dim_B, int *m, int * x_path, int * y_path){
//void wrapper_FindPartition(int *a, int dim_A, int *b, int dim_B, int *m){
  int *A_GPU, *B_GPU, *M_GPU, *X_GPU, *Y_GPU;

  int count;
  cudaDeviceProp prop;
  testCUDA(cudaGetDeviceCount(&count));
  testCUDA(cudaGetDeviceProperties(&prop, count-1));
  printf("Global memory size in octet (bytes): %ld \n", prop.totalGlobalMem); //ld: long double
  printf("Max dimension of the two input arrays: %ld \n", prop.totalGlobalMem/(sizeof(int)*8)); 
  printf("Shared memory size per block: %ld \n", prop.sharedMemPerBlock);
  printf("Max dimension of the int array in the shared memory: %ld \n", prop.sharedMemPerBlock/(sizeof(int)*8)); 
  printf("Number of multiprocessors: %i \n", prop.multiProcessorCount); //NSM
  printf("Maximum number of thread per block: %li \n", prop.maxThreadsPerBlock);
  //3*2^14 -> 3*2^11

  float TimerV;
  cudaEvent_t start, stop;
  testCUDA(cudaEventCreate(&start));
  testCUDA(cudaEventCreate(&stop));
  testCUDA(cudaEventRecord(start,0));

  testCUDA(cudaMalloc(&A_GPU, dim_A*sizeof(int)));
  testCUDA(cudaMalloc(&B_GPU, dim_B*sizeof(int)));
  testCUDA(cudaMalloc(&X_GPU, (dim_A+dim_B+1)*sizeof(int)));
  testCUDA(cudaMalloc(&Y_GPU, (dim_A+dim_B+1)*sizeof(int)));
  testCUDA(cudaMalloc(&M_GPU, (dim_A+dim_B)*sizeof(int)));
  //Copying the values form one ProcUni to the other ProcUnit
  testCUDA(cudaMemcpy(A_GPU, a, dim_A*sizeof(int), cudaMemcpyHostToDevice));
  testCUDA(cudaMemcpy(B_GPU, b, dim_B*sizeof(int), cudaMemcpyHostToDevice));

  printf("Max number of blocks available: %li \n",prop.maxGridSize[0] );

  printf("Number of blocks used: %i \n", NSM);
  printf("Number of thread per block: %i \n", Z);
  MergePath_partition<<<NSM,1>>>(A_GPU, dim_A, B_GPU, dim_B, X_GPU, Y_GPU);

  pathBig_k<<<NSM,Z>>>(X_GPU, Y_GPU, A_GPU, B_GPU, dim_A, dim_B);

  mergeBig_k<<<NSM,Z>>>(X_GPU, Y_GPU, A_GPU, B_GPU, M_GPU, dim_A, dim_B);
  //Copying the value from one ProcUnit ot the ohter ProcUnit
    testCUDA(cudaMemcpy(x_path, X_GPU,(dim_A+dim_B+1)*sizeof(int), cudaMemcpyDeviceToHost));
    testCUDA(cudaMemcpy(y_path, Y_GPU,(dim_A+dim_B+1)*sizeof(int), cudaMemcpyDeviceToHost));
    testCUDA(cudaMemcpy(m, M_GPU, (dim_A+dim_B)*sizeof(int), cudaMemcpyDeviceToHost));

  testCUDA(cudaEventRecord(stop,0));
  testCUDA(cudaEventSynchronize(stop));
  testCUDA(cudaEventElapsedTime(&TimerV, start, stop));
  printf("Execution time of FIND PATH: %f ms\n", TimerV);

  //Freeing the GPU MEMORY
  testCUDA(cudaFree(A_GPU));
  testCUDA(cudaFree(B_GPU));
  testCUDA(cudaFree(M_GPU));
  testCUDA(cudaFree(X_GPU));
  testCUDA(cudaFree(Y_GPU));
  testCUDA(cudaEventDestroy(start));
  testCUDA(cudaEventDestroy(stop));
}


//############################# MAIN ###################################
int main(){
  //INITIALIZATION:
  
  int A[] = {1,2,5,6,6,9,11,15,16};// Project exmaple
  int B[] = {4,7,8,10,12,13,14};// Project exmaple
  int dim_A = 9;// Project exmaple, A is an array with 9 elements
  int dim_B = 7;// Project exmaple, B is an array with 7 elements
  // The Correct mergesort Path is supposed to be 
  int  *X_path, *Y_path, *M; //*A, *B,
  //int *A, *B, *M;
  //int d = 20;
  srand (time(NULL));
  //int dim_A = rand() % d; // !!!
  //int dim_B = d - dim_A;
  //if(dim_B > dim_A)
    //std::swap(dim_A,dim_B);
  //A = (int*)malloc(dim_A*sizeof(int));
  //B = (int*)malloc(dim_B*sizeof(int));
  //#diagonals == #elements chemin: |A|+|B|+1
  X_path = (int*)malloc((dim_A+dim_B+1)*sizeof(int)); // B <-> x
  Y_path = (int*)malloc((dim_A+dim_B+1)*sizeof(int)); // A <-> -y
  //generate_random(A,dim_A,2,98);
  mergesort(A,0,dim_A-1);
  //A[0] = 1;
  //A[dim_A-1]=100;
  //generate_random(B,dim_B,2,98);
  mergesort(B,0,dim_B-1);
  //B[0] = 0;
  //B[dim_B-1] = 101;
  assert((dim_A+dim_B)<=1024); //Check: |M| <= 1024
  M = (int*)malloc((dim_A+dim_B)*sizeof(int));

  // Necessary condition: |A| > |B| !!!
    if(dim_A<dim_B)
      exchange(A, B, dim_A, dim_B);

  printf("A: \n");
  print_array(A,dim_A);

  printf("B: \n");
  print_array(B,dim_B);

    wrapper_FindPartition(A, dim_A, B, dim_B, M, X_path, Y_path);
 // wrapper_FindPartition(A, dim_A, B, dim_B, M);

  printf("M: \n");
  print_array(M, dim_A+dim_B);


    printf("Path: \n");
    for (int i=0; i<5; ++i)
    printf("X[%i]=%i Y[%i]=%i \n",i,X_path[i],i,Y_path[i]);
    printf("\n ... \n");
    for(int i=(dim_A+dim_B-5); i<(dim_A+dim_B+1);++i)
    //for(int i=max(0,dim_A+dim_B-5); i<(dim_A+dim_B+1); ++i)
    printf("X[%i]=%i Y[%i]=%i \n",i,X_path[i],i,Y_path[i]);
    printf("Path: \n");
    for (int i=0; i<dim_A+dim_B+1; ++i)
    printf("X[%i]=%i Y[%i]=%i \n",i,X_path[i],i,Y_path[i]); 
    printf("\n");

  //free(A);
  //free(B);
  free(M);
  free(X_path);
  free(Y_path);
}



'File written in /content/src/main.cu'

In [ ]:
!cd src && make

nvcc -lineinfo main.cu -o main  
main.cu(27): warning: variable "P_x" was set but never used

main.cu(77): warning: variable "P_x" was set but never used

main.cu(146): warning: variable "A_sh" was set but never used

main.cu(146): warning: variable "B_sh" was set but never used

main.cu: In function ‘void wrapper_FindPartition(int*, int, int*, int, int*, int*, int*)’:
main.cu:193:77: warning: format ‘%li’ expects argument of type ‘long int’, but argument 2 has type ‘int’ [-Wformat=]
   printf("Maximum number of thread per block: %li \n", prop.maxThreadsPerBlock);
                                                      ~~~~~~~~~~~~~~~~~~~~~~~^
main.cu:211:71: warning: format ‘%li’ expects argument of type ‘long int’, but argument 2 has type ‘int’ [-Wformat=]
   printf("Max number of blocks available: %li \n",prop.maxGridSize[0] );
                                                  ~~~~~~~~~~~~~~~~~~~~~^


In [ ]:
!cd src && make run 

./main
A: 
dim: 9 
1 2 5 6 6 9 11 15 16 
B: 
dim: 7 
4 7 8 10 12 13 14 
Global memory size in octet (bytes): 17071734784 
Max dimension of the two input arrays: 533491712 
Shared memory size per block: 49152 
Max dimension of the int array in the shared memory: 1536 
Number of multiprocessors: 56 
Maximum number of thread per block: 1024 
Max number of blocks available: 2147483647 
Number of blocks used: 4096 
Number of thread per block: 32 
X[0] = 3, Y[0] = 5X[0] = 4, Y[0] = 7X[0] = 7, Y[0] = 7X[0] = 0, Y[0] = 1X[0] = 1, Y[0] = 5X[0] = 3, Y[0] = 6X[0] = 1, Y[0] = 2X[0] = 1, Y[0] = 4X[0] = 6, Y[0] = 7X[0] = 2, Y[0] = 5X[0] = 0, Y[0] = 2X[0] = 4, Y[0] = 6X[0] = 1, Y[0] = 3X[0] = 7, Y[0] = 8X[0] = 5, Y[0] = 7X[0] = 7, Y[0] = 9Execution time of FIND PATH: 0.519104 ms
M: 
dim: 16 
1 1 1 5 1 1 7 1 1 10 1 1 13 1 1 16 
Path: 
X[0]=0 Y[0]=0 
X[1]=0 Y[1]=1 
X[2]=0 Y[2]=2 
X[3]=1 Y[3]=2 
X[4]=1 Y[4]=3 

 ... 
X[11]=4 Y[11]=7 
X[12]=5 Y[12]=7 
X[13]=6 Y[13]=7 
X[14]=7 Y[14]=7 
X[15]=7 Y[15]=8 
X[

# Execution time algorithm 1.2 


In [ ]:
%%cuda --name main.cu
#include<stdio.h>
#include<math.h>
#include<cassert>
#include<time.h>
#include<algorithm>
#include"repos/functions/functions.hpp"
#define testCUDA(error) (testCUDA(error,__FILE__,__LINE__))

#define NSM 4096 //optimal for execution time 
// 56 = number of multiprocessors -> prop.multiProcessorCount
#define Z 32

//############ KERNEL - find MERGE PATH in the PARTITION POINTS ##############
//X_path -> horizontal (>) <-> "B"-direction
//Y_path -> vertical (v) <-> "A"-direction
__global__ void MergePath_partition(int *A_gpu, int dim_A, int *B_gpu, int dim_B, int *X_gpu, int *Y_gpu){
//Divide the work among the processors: PARTITIONING
//"idx" is the PARTITIONS: it goes from 0 to #partitions (=#NSM)
  int idx = min(blockIdx.x * (dim_A+dim_B) / NSM, dim_A+dim_B);
  int K_x, K_y, P_x, P_y, Q_x, Q_y;
  int off_set;

  //Cross diagonal binary search for MERGE PATH at the PARTITION POINTS on GLOBAL MEMORY
  //Writing the MERGE PATH at PARTITION POINTS on the GLOBAL MEMORY
    if(idx>dim_A){
      K_x = idx - dim_A; //(K_x, K_y): low point of diagonal
      K_y = dim_A;
      P_x = dim_A;      //(P_x,P_y): high point of diagonal
      P_y = idx - dim_A;
    }else{
      K_x = 0;
      K_y = idx;
      P_x = idx;
      P_y = 0;
    }
    while(1){
      off_set = (abs(K_y-P_y)/2);// integer
      Q_x = K_x + off_set;
      Q_y = K_y - off_set;
      if (Q_y>=0 && Q_x<=dim_B && (Q_y == dim_A || Q_x==0 || A_gpu[Q_y] > B_gpu[Q_x-1]) ){
      // A[Q_y]>B[Q_x-1] -> the "path" goes 'over Q' or 'pass through Q'
          if (Q_x == dim_B || Q_y == 0 || A_gpu[Q_y-1] <= B_gpu[Q_x]){
          // A[Q_y-1]<=B[Q_x] -> the "path" goes 'under Q' or 'pass through Q' -> Q is on the "path"
                X_gpu[idx] = Q_x;
                Y_gpu[idx] = Q_y;
                break;
          }else{
              K_x = Q_x + 1;
              K_y = Q_y - 1;
          }
      }else{
        P_x = Q_x - 1;
        P_y = Q_y + 1;
      }
    }
  //The Merge Path terminates in the point (dim_B, dim_A):
  if(blockIdx.x==0 && threadIdx.x==0){
    X_gpu[dim_A+dim_B] = dim_B;
    Y_gpu[dim_A+dim_B] = dim_A;
  }
}


//######################### KERNEL - BIG PATH #################################
__global__ void pathBig_k(int *X_gpu, int *Y_gpu, const int *A_gpu, const int *B_gpu, int dim_A, int dim_B){
  //idx_start, idx_end: beginning of the partition, end of the partition
  int idx_start = min(blockIdx.x * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int idx_end = min((blockIdx.x+1) * (dim_A+dim_B) / NSM, dim_A + dim_B);
  //x_start, y_start: coordinates of the first element of the MergePath in the partition
  int x_start = X_gpu[idx_start];
  int y_start = Y_gpu[idx_start];

  //Place Z elements from each input array in the SHARED MEMORY for each partition
  __shared__ int A_sh[Z], B_sh[Z];
  int idx_sh = threadIdx.x;
  int K_x, K_y, P_x, P_y, Q_x, Q_y;
  int off_set;

//Cross diagonal binary search for MERGE PATH on SHARED MEMORY
//Writing the MERGE PATH on the GLOBAL MEMORY
 while(idx_start + idx_sh < idx_end && idx_sh < Z){
   //Placing the input arrays in the SHARED MEMORY
  if(idx_sh + y_start < dim_A) //in order to have valid indexes for "A_sh" and "A_gpu"
    A_sh[idx_sh] = A_gpu[idx_sh + y_start];
  if(idx_sh + x_start < dim_B)
    B_sh[idx_sh] = B_gpu[idx_sh + x_start];

    if(idx_sh + y_start + 1 > dim_A){
      //since we know already the Merge Path at the starting point: (x_start, y_start)
      //we want to look for the Merge Path from the "next diagonal", that's why "+1"
      K_x = x_start + (idx_sh + y_start + 1 - dim_A);
      K_y = dim_A;
      } else{
        K_x = x_start;
        K_y = idx_sh + y_start + 1;
        }
    if(idx_sh + x_start + 1 > dim_B){
      P_x = dim_B;
      P_y = y_start + (idx_sh + x_start + 1 - dim_B);
      } else{
        P_x = idx_sh + x_start + 1;
        P_y = y_start;
      }

      while(1){
      off_set = (abs(K_y-P_y)/2);
      Q_x = K_x + off_set;
      Q_y = K_y - off_set;
      if ((Q_y>=y_start && Q_y>=0) && (Q_x<=(Z+x_start)<=dim_B) && ((Q_y == (Z+y_start) || Q_y == dim_A) || (Q_x==x_start || Q_x==0) || A_sh[Q_y - y_start] > B_sh[Q_x -1 - x_start]) ){
      // A[Q_y>B[Q_x-1] -> the "path" goes 'over Q' or 'pass through Q'
          if ((Q_x == (Z+x_start) || Q_x == dim_B) || (Q_y == y_start || Q_y == 0) || A_sh[Q_y -1 - y_start] <= B_sh[Q_x - x_start]){
                X_gpu[idx_start + idx_sh + 1] = Q_x;
                Y_gpu[idx_start + idx_sh + 1] = Q_y;

                break;
          }else{
              K_x = Q_x + 1;
              K_y = Q_y - 1;
          }
      }else{
        P_x = Q_x - 1;
        P_y = Q_y + 1;
      }
    }

    //Updating the indexes:
    x_start = X_gpu[idx_start + Z]; //x_start = x_end
    y_start = Y_gpu[idx_start + Z]; //y_start = y_end
    idx_start += Z;
  }
}

//######################### KERNEL to MERGE  ##################################
__global__ void mergeBig_k(int *X_gpu, int *Y_gpu, int *A_gpu, int *B_gpu, int *M_gpu, int dim_A, int dim_B){

  int idx_start = min(blockIdx.x * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int idx_end = min((blockIdx.x+1) * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int x_start = X_gpu[idx_start];
  int y_start = Y_gpu[idx_start];

  //Place Z ponits of the MERGE PATH in the SHARED MEMORY
  __shared__ int X_sh[Z], Y_sh[Z];

  int idx_sh = threadIdx.x;

 while(idx_start + idx_sh < idx_end && idx_sh < Z){
   //Placing the MERGE PATH in the SHARED MEMORY
    X_sh[idx_sh] = X_gpu[idx_start + idx_sh + 1];
    Y_sh[idx_sh] = Y_gpu[idx_start + idx_sh + 1];

    if(idx_sh == 0 && X_sh[0] > x_start) //If the path is moving right
      M_gpu[idx_start] = B_gpu[x_start];
    else if(idx_sh ==0) //if the path is moving down
      M_gpu[idx_start] = A_gpu[y_start];

    if( idx_sh> 0 && X_sh[idx_sh]>X_sh[idx_sh-1]) //If the path is moving right
      M_gpu[idx_start + idx_sh] = B_gpu[X_sh[idx_sh -1]];
    else if(idx_sh > 0) //if the path is moving down
      M_gpu[idx_start + idx_sh] = A_gpu[Y_sh[idx_sh -1]];

    //Update the indexes
    x_start = X_sh[Z-1];
    y_start = Y_sh[Z-1];
    idx_start += Z;
  }
}
//########################### WRAPPER FIND PATH ###################################
void wrapper_MergeSort(int *a, int dim_A, int *b, int dim_B, int *m, float & duration){
  int *A_GPU, *B_GPU, *M_GPU, *X_GPU, *Y_GPU;

  int count;
  cudaDeviceProp prop;
  testCUDA(cudaGetDeviceCount(&count));
  testCUDA(cudaGetDeviceProperties(&prop, count-1));

  //Initilizating the timer:
  float TimerV;
  cudaEvent_t start, stop;
  testCUDA(cudaEventCreate(&start));
  testCUDA(cudaEventCreate(&stop));
  testCUDA(cudaEventRecord(start,0));

  //Initialization on GPU:
  testCUDA(cudaMalloc(&A_GPU, dim_A*sizeof(int)));
  testCUDA(cudaMalloc(&B_GPU, dim_B*sizeof(int)));
  testCUDA(cudaMalloc(&X_GPU, (dim_A+dim_B+1)*sizeof(int)));
  testCUDA(cudaMalloc(&Y_GPU, (dim_A+dim_B+1)*sizeof(int)));
  testCUDA(cudaMalloc(&M_GPU, (dim_A+dim_B)*sizeof(int)));

  testCUDA(cudaMemcpy(A_GPU, a, dim_A*sizeof(int), cudaMemcpyHostToDevice));
  testCUDA(cudaMemcpy(B_GPU, b, dim_B*sizeof(int), cudaMemcpyHostToDevice));

  //##### KERNEL CALLS ######
  MergePath_partition<<<NSM,1>>>(A_GPU, dim_A, B_GPU, dim_B, X_GPU, Y_GPU);

  pathBig_k<<<NSM,Z>>>(X_GPU, Y_GPU, A_GPU, B_GPU, dim_A, dim_B);

  mergeBig_k<<<NSM,Z>>>(X_GPU, Y_GPU, A_GPU, B_GPU, M_GPU, dim_A, dim_B);

  //Bringing back the results from GPU to CPU:
  testCUDA(cudaMemcpy(m, M_GPU, (dim_A+dim_B)*sizeof(int), cudaMemcpyDeviceToHost));

  //Stop the timer:
  testCUDA(cudaEventRecord(stop,0));
  testCUDA(cudaEventSynchronize(stop));
  testCUDA(cudaEventElapsedTime(&TimerV, start, stop));
  printf("Execution time: %f\n \n", TimerV);
  duration = TimerV;

  //Freeing the GPU MEMORY
  testCUDA(cudaFree(A_GPU));
  testCUDA(cudaFree(B_GPU));
  testCUDA(cudaFree(M_GPU));
  testCUDA(cudaFree(X_GPU));
  testCUDA(cudaFree(Y_GPU));
  testCUDA(cudaEventDestroy(start));
  testCUDA(cudaEventDestroy(stop));
}


//############################# MAIN ###################################
int main(){
  //Initialization
  int *A, *B, *M;
  int d[20];
  float duration[20];
  for(int i=0; i<20; ++i)
    d[i] = pow(2,i+1); //2, 4, 8, 16, ..., 1.000.000
  d[19] = 1e6;

  for(int k=0; k<20; k++){
    srand (time(NULL));
    int dim_A = rand() % (d[k]-1) + 1; //choosing randomly the dimension
    int dim_B = d[k] - dim_A;
    if(dim_B > dim_A) //necessary condition: |A| > |B|
      std::swap(dim_A,dim_B);
    A = (int*)malloc(dim_A*sizeof(int));
    B = (int*)malloc(dim_B*sizeof(int));
    generate_random(A,dim_A,0,1000); //random generation of the input arrays
    A[dim_A/2] = 1000;
    mergesort(A,0,dim_A-1);
    generate_random(B,dim_B,0,1000);
    B[dim_B/2] = -1;
    mergesort(B,0,dim_B-1);
    M = (int*)malloc((dim_A+dim_B)*sizeof(int));

    
    printf("%i)\nA: \n",k);
    print_array(A,dim_A);
    printf("\n");
    
    printf("B: \n");
    print_array(B,dim_B);
    printf("\n");

    //###### WRAPPER CALL ######
    wrapper_MergeSort(A, dim_A, B, dim_B, M, duration[k]);


    
    printf("M: \n");
    print_array(M, dim_A+dim_B);
    printf("\n \n \n");

    free(A);
    free(B);
    free(M);
  }
}

 

UsageError: Cell magic `%%cuda` not found.


In [ ]:
!cd src && make

nvcc -lineinfo main.cu -o main  
main.cu(20): warning: variable "P_x" was set but never used

main.cu(76): warning: variable "P_x" was set but never used



In [ ]:
!cd src && make run

./main
0)
A: 
dim: 1 
1000 

B: 
dim: 1 
-1 

Execution time: 0.273760
 
M: 
dim: 2 
-1 1000 

 
 
1)
A: 
dim: 3 
121 257 1000 

B: 
dim: 1 
-1 

Execution time: 0.209344
 
M: 
dim: 4 
-1 121 257 1000 

 
 
2)
A: 
dim: 5 
257 482 609 983 1000 

B: 
dim: 3 
-1 536 770 

Execution time: 0.193408
 
M: 
dim: 8 
-1 257 482 536 609 770 983 1000 

 
 
3)
A: 
dim: 12 
93 121 257 273 482 536 605 609 618 770 983 1000 

B: 
dim: 4 
-1 300 591 698 

Execution time: 0.188800
 
M: 
dim: 16 
-1 93 121 257 273 300 482 536 591 605 609 618 698 770 983 1000 

 
 
4)
A: 
dim: 27 
93 111 121 190 207 233 257 273 300 308  ... 604 605 609 618 653 684 770 875 983 1000 

B: 
dim: 5 
-1 339 626 708 948 

Execution time: 0.164832
 
M: 
dim: 32 
-1 93 111 121 190 207 233 257 273 300  ... 618 626 653 684 708 770 875 948 983 1000 

 
 
5)
A: 
dim: 52 
44 90 93 100 111 121 190 207 233 257  ... 708 770 779 870 875 933 939 948 983 1000 

B: 
dim: 12 
-1 145 197 315 316 506 511 522 561 794 837 881 

Execution time: 0.17

# Question 1.3 : Parallel MergeSort 

In [ ]:
%%cuda --name main.cu
/* Looping on appropriate calls of "pathBig_k" and "mergeBig_k", write a function that sorts any array "M" of size "d" sufficiently smaller than the global memory. Give the execution time with respect to "d" */

#include<stdio.h>
#include<math.h>
#include<cassert>
#include<time.h>
#include<fstream> //to export data
#include<algorithm>
#include"repos/functions/functions.hpp"
#define testCUDA(error) (testCUDA(error,__FILE__,__LINE__))

#define NSM 4096
#define Z 32

//#################### KERNEL - find MERGE PATH in the PARTITION POINTS ##############################
__global__ void MergePath_partition(int *A_gpu, int dim_A, int *B_gpu, int dim_B, int *X_gpu, int *Y_gpu){
  int idx = min(blockIdx.x * (dim_A+dim_B) / NSM, dim_A+dim_B);
  int K_x, K_y, P_x, P_y, Q_x, Q_y;
  int off_set;

    if(idx>dim_A){
      K_x = idx - dim_A;
      K_y = dim_A;
      P_x = dim_A;
      P_y = idx - dim_A;
    }else{
      K_x = 0;
      K_y = idx;
      P_x = idx;
      P_y = 0;
    }
    while(1){
      off_set = (abs(K_y-P_y)/2);//integer
      Q_x = K_x + off_set;
      Q_y = K_y - off_set;
      if (Q_y>=0 && Q_x<=dim_B && (Q_y == dim_A || Q_x==0 || A_gpu[Q_y] > B_gpu[Q_x-1]) ){
          if (Q_x == dim_B || Q_y == 0 || A_gpu[Q_y-1] <= B_gpu[Q_x]){
                X_gpu[idx] = Q_x;
                Y_gpu[idx] = Q_y;
                break;
          }else{
              K_x = Q_x + 1;
              K_y = Q_y - 1;
          }
      }else{
        P_x = Q_x - 1;
        P_y = Q_y + 1;
      }
    }
  if(blockIdx.x==0 && threadIdx.x==0){
    X_gpu[dim_A+dim_B] = dim_B;
    Y_gpu[dim_A+dim_B] = dim_A;
  }
}


//######################### KERNEL - BIG PATH #################################
__global__ void pathBig_k(int *X_gpu, int *Y_gpu, const int *A_gpu, const int *B_gpu, int dim_A, int dim_B){
  int idx_start = min(blockIdx.x * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int idx_end = min((blockIdx.x+1) * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int x_start = X_gpu[idx_start];
  int y_start = Y_gpu[idx_start];

  __shared__ int A_sh[Z], B_sh[Z];
  int idx_sh = threadIdx.x;
  int K_x, K_y, P_x, P_y, Q_x, Q_y;
  int off_set;

 while(idx_start + idx_sh < idx_end && idx_sh < Z){
  if(idx_sh + y_start < dim_A)
    A_sh[idx_sh] = A_gpu[idx_sh + y_start];
  if(idx_sh + x_start< dim_B)
    B_sh[idx_sh] = B_gpu[idx_sh + x_start];
    if(idx_sh + y_start + 1 > dim_A){
      K_x = x_start + (idx_sh + y_start + 1 - dim_A);
      K_y = dim_A;
      } else{
        K_x = x_start;
        K_y = idx_sh + y_start + 1;
        }
    if(idx_sh + x_start + 1 > dim_B){
      P_x = dim_B;
      P_y = y_start + (idx_sh + x_start + 1 - dim_B);
      } else{
        P_x = idx_sh + x_start + 1;
        P_y = y_start;
      }

      while(1){
      off_set = (abs(K_y-P_y)/2);// integer
      Q_x = K_x + off_set;
      Q_y = K_y - off_set;
      if ((Q_y>=y_start && Q_y>=0) && (Q_x<=(Z+x_start)<=dim_B) && ((Q_y == (Z+y_start) || Q_y == dim_A) || (Q_x==x_start || Q_x==0) || A_sh[Q_y - y_start] > B_sh[Q_x -1 - x_start]) ){
          if ((Q_x == (Z+x_start) || Q_x == dim_B) || (Q_y == y_start || Q_y == 0) || A_sh[Q_y -1 - y_start] <= B_sh[Q_x - x_start]){
                X_gpu[idx_start + idx_sh +1] = Q_x;
                Y_gpu[idx_start + idx_sh +1] = Q_y;
                break;
          }else{
              K_x = Q_x + 1;
              K_y = Q_y - 1;
          }
      }else{
        P_x = Q_x - 1;
        P_y = Q_y + 1;
      }
    }
    x_start = X_gpu[idx_start + Z];
    y_start = Y_gpu[idx_start + Z];
    idx_start += Z;
  }
}

//######################### KERNEL to MERGE  ##################################
__global__ void mergeBig_k(int *X_gpu, int *Y_gpu, int *A_gpu, int *B_gpu, int *M_gpu, int dim_A, int dim_B){

  int idx_start = min(blockIdx.x * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int idx_end = min((blockIdx.x+1) * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int x_start = X_gpu[idx_start];
  int y_start = Y_gpu[idx_start];

  __shared__ int X_sh[Z], Y_sh[Z];
  int idx_sh = threadIdx.x;

 while(idx_start + idx_sh < idx_end && idx_sh < Z){
    X_sh[idx_sh] = X_gpu[idx_start + idx_sh + 1];
    Y_sh[idx_sh] = Y_gpu[idx_start + idx_sh + 1];
    if(idx_sh == 0 && X_sh[0] > x_start)
      M_gpu[idx_start] = B_gpu[x_start];
    else if(idx_sh ==0)
      M_gpu[idx_start] = A_gpu[y_start];

    if( idx_sh>0 && X_sh[idx_sh]>X_sh[idx_sh-1])
      M_gpu[idx_start + idx_sh] = B_gpu[X_sh[idx_sh -1]];
    else if(idx_sh > 0)
      M_gpu[idx_start + idx_sh] = A_gpu[Y_sh[idx_sh -1]];

    x_start = X_sh[Z-1];
    y_start = Y_sh[Z-1];
    idx_start += Z;
  }
}
//########################### WRAPPER FIND PATH ###################################
void wrapper_FindPartition(int *a, int dim_A, int *b, int dim_B, int *m){
  int *A_GPU, *B_GPU, *M_GPU, *X_GPU, *Y_GPU;

  testCUDA(cudaMalloc(&A_GPU, dim_A*sizeof(int)));
  testCUDA(cudaMalloc(&B_GPU, dim_B*sizeof(int)));
  testCUDA(cudaMalloc(&X_GPU, (dim_A+dim_B+1)*sizeof(int)));
  testCUDA(cudaMalloc(&Y_GPU, (dim_A+dim_B+1)*sizeof(int)));
  testCUDA(cudaMalloc(&M_GPU, (dim_A+dim_B)*sizeof(int)));

  testCUDA(cudaMemcpy(A_GPU, a, dim_A*sizeof(int), cudaMemcpyHostToDevice));
  testCUDA(cudaMemcpy(B_GPU, b, dim_B*sizeof(int), cudaMemcpyHostToDevice));

  MergePath_partition<<<NSM,1>>>(A_GPU, dim_A, B_GPU, dim_B, X_GPU, Y_GPU);

  pathBig_k<<<NSM,Z>>>(X_GPU, Y_GPU, A_GPU, B_GPU, dim_A, dim_B);

  mergeBig_k<<<NSM,Z>>>(X_GPU, Y_GPU, A_GPU, B_GPU, M_GPU, dim_A, dim_B);

  testCUDA(cudaMemcpy(m, M_GPU, (dim_A+dim_B)*sizeof(int), cudaMemcpyDeviceToHost));

  testCUDA(cudaFree(A_GPU));
  testCUDA(cudaFree(B_GPU));
  testCUDA(cudaFree(M_GPU));
  testCUDA(cudaFree(X_GPU));
  testCUDA(cudaFree(Y_GPU));
}

//####################### PARALLEL MERGE SORT ############################
int * merge_parallel (int *T , int p, int q, int r){
  //A: fisrt array, B: second array, M_suport: mergesorted array
  int *A, *B, *M_support;
  int dim_A = q-p+1;
  int dim_B = r-q;
  if(dim_B > dim_A)
    std::swap(dim_A,dim_B);
  A = (int*)malloc(dim_A*sizeof(int));
  B = (int*)malloc(dim_B*sizeof(int));
  for(int i=0, j=p; i<q-p+1; ++i, ++j) //I copy in A the first array
    A[i] = T[j];
  for(int i=0, j=q+1; i<r-q; ++i, ++j) //I copy in B the second array
    B[i] = T[j];
  M_support = (int*)malloc((dim_A+dim_B)*sizeof(int));

//###### WRAPPER CALL #######
  wrapper_FindPartition(A, dim_A, B, dim_B, M_support);

  for(int i=p, j=0; i<=r; ++i, ++j) //I copy the sorted array into the original one
    T[i]=M_support[j];

  free(A);
  free(B);
  free(M_support);
  return T;
}

int * mergesort_parallel(int* T,int p,int r){
  if(p<r){ //It stops when p == r -> if(p!=r) -> it breaks in "SINGLETON"
//if(r-p<1e6)
    int q = floor((p+r)/2); //It divides the block {p,p+1,..,r} into two sub-blocks: {p,..,q} e {q+1,..,r}
    //I sort recursively the two sub-blocks
    //until I get two singletons (p==r): I SORT the SINGLETONS and then I FUSION them
    mergesort_parallel(T,p,q);
    mergesort_parallel(T,q+1,r);
    merge_parallel(T,p,q,r);
  }
  return T;
}

void parallel_sort(int* T, int N){
  mergesort_parallel(T,0,N-1);
}

//############################# MAIN ###################################
int main(){
//###### INITIALIZATION ######
  int *M;
  int d[20];
  float duration[20];
  for(int i=0; i<20; ++i)
    d[i] = pow(2,i+1);
  d[19] = 1e6;

  for(int k=0; k<20; ++k){
  srand (time(NULL));
  M = (int*)malloc(d[k]*sizeof(int));
  generate_random(M,d[k],0,100);
  M[d[k]/2] = -1; //just to check if it sorted correctly
  printf("M %i: \n", k);
  print_array(M, d[k]);

  //Initializating the timer:
  float TimerV;
  cudaEvent_t start, stop;
  testCUDA(cudaEventCreate(&start));
  testCUDA(cudaEventCreate(&stop));


//######### PARALLEL MERGE SORT ##############
  testCUDA(cudaEventRecord(start,0));

  parallel_sort(M,d[k]);

  testCUDA(cudaEventRecord(stop,0));
  testCUDA(cudaEventSynchronize(stop));
  testCUDA(cudaEventElapsedTime(&TimerV, start, stop));
  duration[k] = TimerV;

  printf("M sorted %i: \n", k);
  print_array(M, d[k]);

  printf("\n");
  free(M);

  testCUDA(cudaEventDestroy(start));
  testCUDA(cudaEventDestroy(stop));
}

for(int k=0; k<20; k++)
  printf("Execution time with N = %i: %f ms\n", d[k], duration[k]);

}

'File written in /content/src/main.cu'

In [ ]:
!cd src && make

nvcc -lineinfo main.cu -o main  
main.cu(18): warning: variable "P_x" was set but never used

main.cu(66): warning: variable "P_x" was set but never used



In [ ]:
!cd src && make run

./main
M 0: 
dim: 2 
41 -1 
M sorted 0: 
dim: 2 
-1 41 

M 1: 
dim: 4 
41 62 -1 49 
M sorted 1: 
dim: 4 
-1 41 49 62 

M 2: 
dim: 8 
41 62 6 49 -1 27 26 95 
M sorted 2: 
dim: 8 
-1 6 26 27 41 49 62 95 

M 3: 
dim: 16 
41 62 6 49 0 27 26 95 -1 46 25 79 56 66 50 68 
M sorted 3: 
dim: 16 
-1 0 6 25 26 27 41 46 49 50 56 62 66 68 79 95 

M 4: 
dim: 32 
41 62 6 49 0 27 26 95 73 46  ... 12 96 43 18 81 9 85 34 54 78 
M sorted 4: 
dim: 32 
-1 0 6 9 10 12 14 18 19 25  ... 66 68 73 74 78 79 81 85 95 96 

M 5: 
dim: 64 
41 62 6 49 0 27 26 95 73 46  ... 77 68 84 10 77 21 44 32 51 93 
M sorted 5: 
dim: 64 
-1 0 1 4 6 9 10 10 12 12  ... 79 81 84 85 93 95 96 96 96 99 

M 6: 
dim: 128 
66 44 85 1 23 92 28 11 38 37  ... 2 37 9 43 95 78 29 3 25 82 
M sorted 6: 
dim: 128 
-1 1 1 1 1 1 1 1 2 3  ... 92 93 95 95 95 95 96 96 97 98 

M 7: 
dim: 256 
66 44 85 1 23 92 28 11 38 37  ... 96 52 5 99 87 98 77 85 74 59 
M sorted 7: 
dim: 256 
-1 0 0 1 1 1 1 1 1 1  ... 96 97 97 97 98 98 99 99 99 99 

M 8: 
dim: 512 
66

# Question 2.4 : BATCH MERGE 

In [ ]:
%%cuda --name main.cu
/* In this part, we assume we have a large NUMBER N(>= 10^3) of arrays {A_i}(1<=i<=N) and {B_i}(1<=i<=N)
with |A_i|+|B_i|<=1024 for each i. Using some changes on "mergeSmall_k" we would like to write
"mergeSmallBatch_k" that merges two by two, for each i, A_i and B_i.
Given a fixed common size d<=1024, "mergeSmallBatch_k" is launched using the syntax:
  mergeSmallBatch_k<<<numBlocks, threadsPerBlock>>>(...);
with "threadsPerBlock" is MULTIPLE of "d" but smaller than 1024
and "numBlocks" is an arbirtrary sufficiently big number.
(1) Explain why the indices:
- int tidx = threadIdx.x%d;
    "tidx": since if d<=512 (1024/2 = 512 minimum multiple) we can mergesort more than one array
            per block (exactly m arrays: m = 1024/d). "tidx" allows us to enumerate from 1 to d, the elements of each of pair {A_i},{B_i} with (1<=i<=m) in the block.
- int Qt = (threadIdx.x-tidx)/d;
    "Qt": local numeration of arrays in a block. "Qt" goes from 1 to m
- gbx = Qt + blockIdx.x * (blockDim.x/d);
    "gbx": global numeration of arrays. "gbx" goes from 1 to N
are important in the definition of "mergeSmallBatch_k"
(2) Write the kernel "mergeSmallBatch_k" that batch merges two by two {A_i} and {B_i} (1<=i<=N)
Give the execution time with respect to d = 4,8,...,1024 */

//NTPB => Number of threads per block is a multiple of d !!!

#include<stdio.h>
#include<math.h>
#include<cassert>
#include<time.h>
#include"repos/functions/functions.hpp"
#include<algorithm>
#include<fstream> //to export data
#define testCUDA(error) (testCUDA(error,__FILE__,__LINE__))

#define N 5 // number of arrays

//############################### KERNEL ##################################
__global__ void mergeSmallBatch_k(int **A, int *dim_A, int **B, int *dim_B, int **M, int d){
  int tidx = threadIdx.x % d; //enumeration of elements each array of the block -> 0 : d-1
  int Qt = (threadIdx.x-tidx)/d; //enumeration of the arrays in the block -> 0 : m-1
  int gbx = Qt + blockIdx.x * (blockDim.x/d); //global enumeration of arrays -> 0 : N-1

  //int idx = threadIdx.x;
  int K_x, K_y, P_x, P_y, Q_x, Q_y;
  int off_set;
  if(gbx < N){
    if(tidx>dim_A[gbx]){
      K_x = tidx - dim_A[gbx]; //(K_x, K_y): low point of diagonal
      K_y = dim_A[gbx];
      P_x = dim_A[gbx]; //(P_x,P_y): high point of diagonal
      P_y = tidx - dim_A[gbx];
    }else{
      K_x = 0;
      K_y = tidx;
      P_x = tidx;
      P_y = 0;
    }
    while(1){
      off_set = (abs(K_y-P_y)/2); //integer
      //distance on y == distance on x, because diagonal
      Q_x = K_x + off_set;
      Q_y = K_y - off_set;
      //offset is an "int" (integer), so it's rounded to the smaller integer -> Q will be closer to K
      if (Q_y>=0 && Q_x<=dim_B[gbx] && (Q_y == dim_A[gbx] || Q_x==0 || A[gbx][Q_y] > B[gbx][Q_x-1]) ){
        // (Q_y>=0 and Q_x<=B) -> Q is in the grid
        // Q_y=|A| -> Q is on the down border
        // Q_x=0   -> Q is on the left border
        // A[Q_y>B[Q_x-1] -> the "path" goes 'over Q' or 'pass through Q'
          if (Q_x == dim_B[gbx] || Q_y == 0 || A[gbx][Q_y-1] <= B[gbx][Q_x]){
            // Q_x=|B| -> Q is on the right border
            // Q_y=0   -> Q in on the up border
            // A[Q_y-1]<=B[Q_x] -> the "path" goes 'under Q' or 'pass through Q'
              if (Q_y<dim_A[gbx] && (Q_x == dim_B[gbx] || A[gbx][Q_y]<= B[gbx][Q_x]) )
              //if Q is not on the down border (= if the "path" can go down) and it MUST go down
                M[gbx][tidx] = A[gbx][Q_y]; //the "path" goes down (in A direction)
              else //if it can't go down
                M[gbx][tidx] = B[gbx][Q_x]; //the "path" goes right (in B direction)s

              break;
          }else{ //if the path is over Q but not under Q
              K_x = Q_x + 1; //move Q up by moving K up (updating Q_x to remain on diagonal)
              K_y = Q_y - 1;
          }
      }else{ //if the path is under Q
        P_x = Q_x - 1; //move Q down by moving P down (updating Q_x to remain on diagonal)
        P_y = Q_y + 1;
      }
    }
  }
}


//############################### WRAPPER ##################################
void wrapper(int **A, int *dim_A, int **B, int *dim_B, int **M, int dd){
  //A, B, M: array of N array -> A[i]: array of N elements
  //dim_A, dim_B: array of N integers -> dim_A[i]: dimension of A_i
  int **A_GPU, **B_GPU, **M_GPU;
  int *dim_A_GPU, *dim_B_GPU;

  int count;
  cudaDeviceProp prop;
  testCUDA(cudaGetDeviceCount(&count));
  testCUDA(cudaGetDeviceProperties(&prop, count-1));


  //########## INITIALIZATION ##########
  float TimerV;
  cudaEvent_t start, stop;
  testCUDA(cudaEventCreate(&start));
  testCUDA(cudaEventCreate(&stop));
  testCUDA(cudaEventRecord(start,0));

  testCUDA(cudaMalloc(&dim_A_GPU, N*sizeof(int)));
  testCUDA(cudaMalloc(&dim_B_GPU, N*sizeof(int)));
  testCUDA(cudaMemcpy(dim_A_GPU, dim_A, N*sizeof(int), cudaMemcpyHostToDevice));
  testCUDA(cudaMemcpy(dim_B_GPU, dim_B, N*sizeof(int), cudaMemcpyHostToDevice));

  /*Procedure to allocate on the DEVICE an array of arrays:
  (1)Allocate the POINTERS to a HOST memory
  (2)Allocate DEVICE memory for EACH ARRAY and store its pointer in the host memory
  (3)Allocate DEVICE memory for storing the pointers
  (4)Then copy the host memory to the device memory */

  //(1) Allocate the ARRAY of POINTERS to HOST memory
    // in order to store "A", "B", "M" into "A_GPU", "B_GPU", "M_GPU"
    int ** A_host= (int**)malloc(N*sizeof(int *));
    int ** B_host = (int**)malloc(N*sizeof(int *));
    int ** M_host = (int**)malloc(N*sizeof(int *));
    // in order to copy from "M_ GPU" to "M"
    int ** M_host_pointers = (int**)malloc(N*sizeof(int *));//It is Necessary to keep a copy of the pointers in the host, in order to use cudaMemcpy -> cudaMemcpy(Pointer on CPU of a GPU object, pointer on CPU of a CPU object)

  //(2) Allocate DEVICE memory for EACH ARRAY and store its pointer in the host memory
    for(int i=0; i<N; ++i){
      testCUDA(cudaMalloc(&A_host[i], dim_A[i]*sizeof(int)));
      testCUDA(cudaMalloc(&B_host[i], dim_B[i]*sizeof(int)));
      testCUDA(cudaMalloc(&M_host[i], dd*sizeof(int)));
      M_host_pointers[i] = M_host[i]; //I keep in the host a copy of the pointers
    }
    for(int i=0; i<N; ++i){
      testCUDA(cudaMemcpy(A_host[i], A[i], dim_A[i]*sizeof(int), cudaMemcpyHostToDevice));
      testCUDA(cudaMemcpy(B_host[i], B[i], dim_B[i]*sizeof(int), cudaMemcpyHostToDevice));
    }

  //(3)Allocate DEVICE memory for storing the pointers
    testCUDA(cudaMalloc(&A_GPU, N*sizeof(int *)));
    testCUDA(cudaMalloc(&B_GPU, N*sizeof(int *)));
    testCUDA(cudaMalloc(&M_GPU, N*sizeof(int *)));

  //(4)Then copy the host memory to the device memory
    testCUDA(cudaMemcpy(A_GPU, A_host, N*sizeof(int *), cudaMemcpyHostToDevice));
    testCUDA(cudaMemcpy(B_GPU, B_host, N*sizeof(int *), cudaMemcpyHostToDevice));
    testCUDA(cudaMemcpy(M_GPU, M_host, N*sizeof(int *), cudaMemcpyHostToDevice));


  //########## CALLING THE KERNEL ##########
  printf("N: %i, d: %i \n",N, dd);
  int m = min(1024 / dd, N); //number of pair of arrays that each block can merge
  printf("Number of pair of arrays that each block merge: %i \n", m);
  int NB;// number of blocks needed
  if(N%m == 0)
    NB = N/m;
  else
    NB = N/m + 1;
  printf("Max number of blocks available: %li \n",prop.maxGridSize[0] );
  printf("Max number of block required: %li \n", NB );
  printf("Number of thread per block: %li \n", dd*m);
  if( NB < prop.maxGridSize[0]){
    printf("No need for a while loop \n" );
    mergeSmallBatch_k<<<NB,dd*m>>>(A_GPU, dim_A_GPU, B_GPU, dim_B_GPU, M_GPU, dd);
  } else{
    printf("while loop is needed \n ");
    //Number of blocks: a number smaller than NB_MAX. I chose: NB == NTPB
    mergeSmallBatch_k<<<dd*m,dd*m>>>(A_GPU, dim_A_GPU, B_GPU, dim_B_GPU, M_GPU, dd);
  }

//Copying "M_GPU" into "M", thanks to the HOST pointers in "M_host_pointers"
 for(int i=0; i<N; ++i){
    testCUDA(cudaMemcpy(M[i], M_host_pointers[i], dd*sizeof(int), cudaMemcpyDeviceToHost));   //???
  }


  //##### FREE MEMORY & GET EXECUTION TIME #####
  for(int i=0; i<N; ++i){
    testCUDA(cudaFree(A_host[i]));
    testCUDA(cudaFree(B_host[i]));
    testCUDA(cudaFree(M_host[i]));
  }

  free(A_host);
  free(B_host);
  free(M_host);
  free(M_host_pointers);

  testCUDA(cudaFree(A_GPU));
  testCUDA(cudaFree(B_GPU));
  testCUDA(cudaFree(M_GPU));
  testCUDA(cudaFree(dim_A_GPU));
  testCUDA(cudaFree(dim_B_GPU));

  testCUDA(cudaEventRecord(stop,0));
  testCUDA(cudaEventSynchronize(stop));
  testCUDA(cudaEventElapsedTime(&TimerV, start, stop));
  printf("Execution time: %f ms\n \n", TimerV);

  testCUDA(cudaEventDestroy(start));
  testCUDA(cudaEventDestroy(stop));
}

int main(){
  //#####  INITIALIZATION  #####
 int d = 6;// 6 elements in each 5 arrays mergesorted in M

  int **A, **B, **M; // int **
  int *dim_A, *dim_B;
  A = (int**)malloc(N*sizeof(int *));
  B = (int**)malloc(N*sizeof(int *));
  M = (int**)malloc(N*sizeof(int *));
  dim_A = (int*)malloc(N*sizeof(int));
  dim_B = (int*)malloc(N*sizeof(int));

  srand (time(NULL));
  for(int i=0; i<N; ++i){
      dim_A[i] = rand() % d;
    dim_B[i] = d - dim_A[i];
    if(dim_A[i]<dim_B[i]){
      std::swap(dim_A[i],dim_B[i]);
      std::swap(A[i],B[i]);
    }
    A[i] = (int*)malloc(dim_A[i]*sizeof(int));
    B[i] = (int*)malloc(dim_B[i]*sizeof(int));
    M[i] = (int*)malloc(d*sizeof(int));
    generate_random(A[i],dim_A[i],0,100);
    mergesort(A[i],0,dim_A[i]-1);
    generate_random(B[i],dim_B[i],0,100);
    mergesort(B[i],0,dim_B[i]-1);
  }

  printf("A\n");
  for(int i=0; i<N; ++i){
    printt_array(A[i],dim_A[i]);
    printf("\n");
  }

  printf("B\n");
  for(int i=0; i<N; ++i){
    printt_array(B[i],dim_B[i]);
    printf("\n");
  }


//######## WRAPPER CALL #########
  wrapper(A, dim_A, B, dim_B, M, d);


  printf("M\n");
  for(int i=0; i<N; ++i){
    printt_array(M[i],d);
    printf("\n");
  }


//####### FREE THE MEMORY #######
  for(int i=0; i<N; ++i){
    free(A[i]);
    free(B[i]);
    free(M[i]);
  }


  free(A);
  free(B);
  free(M);
  free(dim_A);
  free(dim_B);

}

'File written in /content/src/main.cu'

In [ ]:
!cd src && make

nvcc -lineinfo main.cu -o main  
main.cu(40): warning: variable "P_x" was set but never used

main.cu: In function ‘void wrapper(int**, int*, int**, int*, int**, int)’:
main.cu:160:71: warning: format ‘%li’ expects argument of type ‘long int’, but argument 2 has type ‘int’ [-Wformat=]
   printf("Max number of blocks available: %li \n",prop.maxGridSize[0] );
                                                  ~~~~~~~~~~~~~~~~~~~~~^
main.cu:161:50: warning: format ‘%li’ expects argument of type ‘long int’, but argument 2 has type ‘int’ [-Wformat=]
   printf("Max number of block required: %li \n", NB );
                                                  ^
main.cu:162:52: warning: format ‘%li’ expects argument of type ‘long int’, but argument 2 has type ‘int’ [-Wformat=]
   printf("Number of thread per block: %li \n", dd*m);
                                              ~~~~~~^


In [ ]:
!cd src && make run

./main
A
36 74 80 93 

1 6 9 29 97 

26 34 47 84 88 

25 47 48 50 75 96 

50 56 65 87 

B
9 91 

33 

51 



10 88 

N: 5, d: 6 
Number of pair of arrays that each block merge: 5 
Max number of blocks available: 2147483647 
Max number of block required: 1 
Number of thread per block: 30 
No need for a while loop 
Execution time: 0.674880 ms
 
M
9 36 74 80 91 93 

1 6 9 29 33 97 

26 34 47 51 84 88 

25 47 48 50 75 96 

10 50 56 65 87 88 



# Question 2.5 : BATCH MERGE Execution time 

In [ ]:
%%cuda --name main.cu
#include<stdio.h>
#include<math.h>
#include<cassert>
#include<time.h>
#include"repos/functions/functions.hpp"
#include<algorithm>
#include<fstream> //to export data
#define testCUDA(error) (testCUDA(error,__FILE__,__LINE__))

#define N 1000000

//############################### KERNEL ##################################
__global__ void mergeSmallBatch_k(int **A, int *dim_A, int **B, int *dim_B, int **M, int d){
  int tidx = threadIdx.x % d; //enumeration of elements each array of the block -> 0 : d-1
  int Qt = (threadIdx.x-tidx)/d; //enumeration of the arrays in the block -> 0 : m-1
  int gbx = Qt + blockIdx.x * (blockDim.x/d); //global enumeration of arrays -> 0 : N-1

  //int idx = threadIdx.x;
  int K_x, K_y, P_x, P_y, Q_x, Q_y;
  int off_set;
  if(gbx < N){
    if(tidx>dim_A[gbx]){
      K_x = tidx - dim_A[gbx]; //(K_x, K_y): low point of diagonal
      K_y = dim_A[gbx];
      P_x = dim_A[gbx]; //(P_x,P_y): high point of diagonal
      P_y = tidx - dim_A[gbx];
    }else{
      K_x = 0;
      K_y = tidx;
      P_x = tidx;
      P_y = 0;
    }
    while(1){
      off_set = (abs(K_y-P_y)/2); //integer
      //distance on y == distance on x, because diagonal
      Q_x = K_x + off_set;
      Q_y = K_y - off_set;
      //offset is an "int" (integer), so it's rounded to the smaller integer -> Q will be closer to K
      if (Q_y>=0 && Q_x<=dim_B[gbx] && (Q_y == dim_A[gbx] || Q_x==0 || A[gbx][Q_y] > B[gbx][Q_x-1]) ){
        // (Q_y>=0 and Q_x<=B) -> Q is in the grid
        // Q_y=|A| -> Q is on the down border
        // Q_x=0   -> Q is on the left border
        // A[Q_y>B[Q_x-1] -> the "path" goes 'over Q' or 'pass through Q'
          if (Q_x == dim_B[gbx] || Q_y == 0 || A[gbx][Q_y-1] <= B[gbx][Q_x]){
            // Q_x=|B| -> Q is on the right border
            // Q_y=0   -> Q in on the up border
            // A[Q_y-1]<=B[Q_x] -> the "path" goes 'under Q' or 'pass through Q'
              if (Q_y<dim_A[gbx] && (Q_x == dim_B[gbx] || A[gbx][Q_y]<= B[gbx][Q_x]) )
              //if Q is not on the down border (= if the "path" can go down) and it MUST go down
                M[gbx][tidx] = A[gbx][Q_y]; //the "path" goes down (in A direction)
              else //if it can't go down
                M[gbx][tidx] = B[gbx][Q_x]; //the "path" goes right (in B direction)s

              break;
          }else{ //if the path is over Q but not under Q
              K_x = Q_x + 1; //move Q up by moving K up (updating Q_x to remain on diagonal)
              K_y = Q_y - 1;
          }
      }else{ //if the path is under Q
        P_x = Q_x - 1; //move Q down by moving P down (updating Q_x to remain on diagonal)
        P_y = Q_y + 1;
      }
    }
  }
}


//############################### WRAPPER ##################################
void wrapper(int **A, int *dim_A, int **B, int *dim_B, int **M, int dd, float *duration, int k){
  //A, B, M: array of N array -> A[i]: array of N elements
  //dim_A, dim_B: array of N integers -> dim_A[i]: dimension of A_i
  int **A_GPU, **B_GPU, **M_GPU;
  int *dim_A_GPU, *dim_B_GPU;

  int count;
  cudaDeviceProp prop;
  testCUDA(cudaGetDeviceCount(&count));
  testCUDA(cudaGetDeviceProperties(&prop, count-1));


  //########## INITIALIZATION ##########
  float TimerV;
  cudaEvent_t start, stop;
  testCUDA(cudaEventCreate(&start));
  testCUDA(cudaEventCreate(&stop));
  testCUDA(cudaEventRecord(start,0));

  testCUDA(cudaMalloc(&dim_A_GPU, N*sizeof(int)));
  testCUDA(cudaMalloc(&dim_B_GPU, N*sizeof(int)));
  testCUDA(cudaMemcpy(dim_A_GPU, dim_A, N*sizeof(int), cudaMemcpyHostToDevice));
  testCUDA(cudaMemcpy(dim_B_GPU, dim_B, N*sizeof(int), cudaMemcpyHostToDevice));

  /*Procedure to allocate on the DEVICE an array of arrays:
  (1)Allocate the POINTERS to a HOST memory
  (2)Allocate DEVICE memory for EACH ARRAY and store its pointer in the host memory
  (3)Allocate DEVICE memory for storing the pointers
  (4)Then copy the host memory to the device memory */

  //(1) Allocate the ARRAY of POINTERS to HOST memory
    // in order to store "A", "B", "M" into "A_GPU", "B_GPU", "M_GPU"
    int ** A_host= (int**)malloc(N*sizeof(int *));
    int ** B_host = (int**)malloc(N*sizeof(int *));
    int ** M_host = (int**)malloc(N*sizeof(int *));
    // in order to copy from "M_ GPU" to "M"
    int ** M_host_pointers = (int**)malloc(N*sizeof(int *));//It is Necessary to keep a copy of the pointers in the host, in order to use cudaMemcpy -> cudaMemcpy(Pointer on CPU of a GPU object, pointer on CPU of a CPU object)

  //(2) Allocate DEVICE memory for EACH ARRAY and store its pointer in the host memory
    for(int i=0; i<N; ++i){
      testCUDA(cudaMalloc(&A_host[i], dim_A[i]*sizeof(int)));
      testCUDA(cudaMalloc(&B_host[i], dim_B[i]*sizeof(int)));
      testCUDA(cudaMalloc(&M_host[i], dd*sizeof(int)));
      M_host_pointers[i] = M_host[i]; //I keep in the host a copy of the pointers
    }
    for(int i=0; i<N; ++i){
      testCUDA(cudaMemcpy(A_host[i], A[i], dim_A[i]*sizeof(int), cudaMemcpyHostToDevice));
      testCUDA(cudaMemcpy(B_host[i], B[i], dim_B[i]*sizeof(int), cudaMemcpyHostToDevice));
    }

  //(3)Allocate DEVICE memory for storing the pointers
    testCUDA(cudaMalloc(&A_GPU, N*sizeof(int *)));
    testCUDA(cudaMalloc(&B_GPU, N*sizeof(int *)));
    testCUDA(cudaMalloc(&M_GPU, N*sizeof(int *)));

  //(4)Then copy the host memory to the device memory
    testCUDA(cudaMemcpy(A_GPU, A_host, N*sizeof(int *), cudaMemcpyHostToDevice));
    testCUDA(cudaMemcpy(B_GPU, B_host, N*sizeof(int *), cudaMemcpyHostToDevice));
    testCUDA(cudaMemcpy(M_GPU, M_host, N*sizeof(int *), cudaMemcpyHostToDevice));


  //########## CALLING THE KERNEL ##########
  printf("N: %i, d: %i \n",N, dd);
  int m = min(1024 / dd, N); //number of pair of arrays that each block can merge
  printf("Number of pair of arrays that each block merge: %i \n", m);
  int NB;// number of blocks needed
  if(N%m == 0)
    NB = N/m;
  else
    NB = N/m + 1;
  printf("Max number of blocks available: %li \n",prop.maxGridSize[0] );
  printf("Max number of block required: %li \n", NB );
  printf("Number of thread per block: %li \n", dd*m);
  if( NB < prop.maxGridSize[0]){
    printf("No need for a while loop \n" );
    mergeSmallBatch_k<<<NB,dd*m>>>(A_GPU, dim_A_GPU, B_GPU, dim_B_GPU, M_GPU, dd);
  } else{
    printf("while loop is needed \n ");
    //Number of blocks: a number smaller than NB_MAX. I chose: NB == NTPB
    mergeSmallBatch_k<<<dd*m,dd*m>>>(A_GPU, dim_A_GPU, B_GPU, dim_B_GPU, M_GPU, dd);
  }

//Copying "M_GPU" into "M", thanks to the HOST pointers in "M_host_pointers"
 for(int i=0; i<N; ++i){
    testCUDA(cudaMemcpy(M[i], M_host_pointers[i], dd*sizeof(int), cudaMemcpyDeviceToHost));   //???
  }


  //##### FREE MEMORY & GET EXECUTION TIME #####
  for(int i=0; i<N; ++i){
    testCUDA(cudaFree(A_host[i]));
    testCUDA(cudaFree(B_host[i]));
    testCUDA(cudaFree(M_host[i]));
  }

  free(A_host);
  free(B_host);
  free(M_host);
  free(M_host_pointers);

  testCUDA(cudaFree(A_GPU));
  testCUDA(cudaFree(B_GPU));
  testCUDA(cudaFree(M_GPU));
  testCUDA(cudaFree(dim_A_GPU));
  testCUDA(cudaFree(dim_B_GPU));

  testCUDA(cudaEventRecord(stop,0));
  testCUDA(cudaEventSynchronize(stop));
  testCUDA(cudaEventElapsedTime(&TimerV, start, stop));
  printf("Execution time: %f ms\n \n", TimerV);
  duration[k] = TimerV;

  testCUDA(cudaEventDestroy(start));
  testCUDA(cudaEventDestroy(stop));
}

int main(){
  //#####  INITIALIZATION  #####
 int d[9];
 float duration[9];
  for(int i=0; i<9; ++i){
    d[i] = pow(2,i+2);
}

  int **A, **B, **M; // int **
  int *dim_A, *dim_B;
  A = (int**)malloc(N*sizeof(int *));
  B = (int**)malloc(N*sizeof(int *));
  M = (int**)malloc(N*sizeof(int *));
  dim_A = (int*)malloc(N*sizeof(int));
  dim_B = (int*)malloc(N*sizeof(int));

for(int k=0; k<9; ++k){
  srand (time(NULL));
  for(int i=0; i<N; ++i){
      dim_A[i] = rand() % d[k];
    dim_B[i] = d[k] - dim_A[i];
    if(dim_A[i]<dim_B[i]){
      std::swap(dim_A[i],dim_B[i]);
      std::swap(A[i],B[i]);
    }
    A[i] = (int*)malloc(dim_A[i]*sizeof(int));
    B[i] = (int*)malloc(dim_B[i]*sizeof(int));
    M[i] = (int*)malloc(d[k]*sizeof(int));
  //  M[i] = (int*)malloc(d*sizeof(int));
    generate_random(A[i],dim_A[i],0,100);
    mergesort(A[i],0,dim_A[i]-1);
    generate_random(B[i],dim_B[i],0,100);
    mergesort(B[i],0,dim_B[i]-1);
  }

/*
  printf("A\n");
  for(int i=0; i<N; ++i){
    print_array(A[i],dim_A[i]);
    printf("\n");
  }
  printf("B\n");
  for(int i=0; i<N; ++i){
    print_array(B[i],dim_B[i]);
    printf("\n");
  }
*/

//######## WRAPPER CALL #########
  wrapper(A, dim_A, B, dim_B, M, d[k], duration, k);


/*
  printf("M\n");
  for(int i=0; i<N; ++i){
    print_array(M[i],d[k]);
    printf("\n");
  }
  */



//####### FREE THE MEMORY #######
  for(int i=0; i<N; ++i){
    free(A[i]);
    free(B[i]);
    free(M[i]);
  }

}
  free(A);
  free(B);
  free(M);
  free(dim_A);
  free(dim_B);

}

'File written in /content/src/main.cu'

In [ ]:
!cd src && make

nvcc -lineinfo main.cu -o main  
main.cu(19): warning: variable "P_x" was set but never used

main.cu: In function ‘void wrapper(int**, int*, int**, int*, int**, int, float*, int)’:
main.cu:139:71: warning: format ‘%li’ expects argument of type ‘long int’, but argument 2 has type ‘int’ [-Wformat=]
   printf("Max number of blocks available: %li \n",prop.maxGridSize[0] );
                                                  ~~~~~~~~~~~~~~~~~~~~~^
main.cu:140:50: warning: format ‘%li’ expects argument of type ‘long int’, but argument 2 has type ‘int’ [-Wformat=]
   printf("Max number of block required: %li \n", NB );
                                                  ^
main.cu:141:52: warning: format ‘%li’ expects argument of type ‘long int’, but argument 2 has type ‘int’ [-Wformat=]
   printf("Number of thread per block: %li \n", dd*m);
                                              ~~~~~~^


In [ ]:
!cd src && make run

./main
N: 1000000, d: 4 
Number of pair of arrays that each block merge: 256 
Max number of blocks available: 2147483647 
Max number of block required: 3907 
Number of thread per block: 1024 
No need for a while loop 
Execution time: 69196.890625 ms
 
N: 1000000, d: 8 
Number of pair of arrays that each block merge: 128 
Max number of blocks available: 2147483647 
Max number of block required: 7813 
Number of thread per block: 1024 
No need for a while loop 
Execution time: 70698.835938 ms
 
N: 1000000, d: 16 
Number of pair of arrays that each block merge: 64 
Max number of blocks available: 2147483647 
Max number of block required: 15625 
Number of thread per block: 1024 
No need for a while loop 
Execution time: 72707.054688 ms
 
N: 1000000, d: 32 
Number of pair of arrays that each block merge: 32 
Max number of blocks available: 2147483647 
Max number of block required: 31250 
Number of thread per block: 1024 
No need for a while loop 
Execution time: 74941.726562 ms
 
N: 1000000,

# Application with Alice and Bob => Cryptography 

In [ ]:
%%cuda --name main.cu 

#include<stdio.h>
#include<math.h>
#include<cassert>
#include<time.h>
#include"repos/functions/functions.hpp"
#include<algorithm>
#include <iostream> 
#include<fstream> //to export data
#define testCUDA(error) (testCUDA(error,__FILE__,__LINE__))

// -----------------------------------------------------------------------------------------------------------------------------------------------------------------
// ----------------------------------------------------- MERGE ALGOS -----------------------------------------------------------------------------------------------
// -----------------------------------------------------------------------------------------------------------------------------------------------------------------

#define NSM 4096
#define Z 32

//#################### KERNEL - find MERGE PATH in the PARTITION POINTS ##############################
__global__ void MergePath_partition(int *A_gpu, int dim_A, int *B_gpu, int dim_B, int *X_gpu, int *Y_gpu){
  int idx = min(blockIdx.x * (dim_A+dim_B) / NSM, dim_A+dim_B);
  int K_x, K_y, P_x, P_y, Q_x, Q_y;
  int off_set;

    if(idx>dim_A){
      K_x = idx - dim_A;
      K_y = dim_A;
      P_x = dim_A;
      P_y = idx - dim_A;
    }else{
      K_x = 0;
      K_y = idx;
      P_x = idx;
      P_y = 0;
    }
    while(1){
      off_set = (abs(K_y-P_y)/2);//integer
      Q_x = K_x + off_set;
      Q_y = K_y - off_set;
      if (Q_y>=0 && Q_x<=dim_B && (Q_y == dim_A || Q_x==0 || A_gpu[Q_y] > B_gpu[Q_x-1]) ){
          if (Q_x == dim_B || Q_y == 0 || A_gpu[Q_y-1] <= B_gpu[Q_x]){
                X_gpu[idx] = Q_x;
                Y_gpu[idx] = Q_y;
                break;
          }else{
              K_x = Q_x + 1;
              K_y = Q_y - 1;
          }
      }else{
        P_x = Q_x - 1;
        P_y = Q_y + 1;
      }
    }
  if(blockIdx.x==0 && threadIdx.x==0){
    X_gpu[dim_A+dim_B] = dim_B;
    Y_gpu[dim_A+dim_B] = dim_A;
  }
}


//######################### KERNEL - BIG PATH #################################
__global__ void pathBig_k(int *X_gpu, int *Y_gpu, const int *A_gpu, const int *B_gpu, int dim_A, int dim_B){
  int idx_start = min(blockIdx.x * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int idx_end = min((blockIdx.x+1) * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int x_start = X_gpu[idx_start];
  int y_start = Y_gpu[idx_start];

  __shared__ int A_sh[Z], B_sh[Z];
  int idx_sh = threadIdx.x;
  int K_x, K_y, P_x, P_y, Q_x, Q_y;
  int off_set;

 while(idx_start + idx_sh < idx_end && idx_sh < Z){
  if(idx_sh + y_start < dim_A)
    A_sh[idx_sh] = A_gpu[idx_sh + y_start];
  if(idx_sh + x_start< dim_B)
    B_sh[idx_sh] = B_gpu[idx_sh + x_start];
    if(idx_sh + y_start + 1 > dim_A){
      K_x = x_start + (idx_sh + y_start + 1 - dim_A);
      K_y = dim_A;
      } else{
        K_x = x_start;
        K_y = idx_sh + y_start + 1;
        }
    if(idx_sh + x_start + 1 > dim_B){
      P_x = dim_B;
      P_y = y_start + (idx_sh + x_start + 1 - dim_B);
      } else{
        P_x = idx_sh + x_start + 1;
        P_y = y_start;
      }

      while(1){
      off_set = (abs(K_y-P_y)/2);//integer
      Q_x = K_x + off_set;
      Q_y = K_y - off_set;
      if ((Q_y>=y_start && Q_y>=0) && (Q_x<=(Z+x_start)<=dim_B) && ((Q_y == (Z+y_start) || Q_y == dim_A) || (Q_x==x_start || Q_x==0) || A_sh[Q_y - y_start] > B_sh[Q_x -1 - x_start]) ){
          if ((Q_x == (Z+x_start) || Q_x == dim_B) || (Q_y == y_start || Q_y == 0) || A_sh[Q_y -1 - y_start] <= B_sh[Q_x - x_start]){
                X_gpu[idx_start + idx_sh +1] = Q_x;
                Y_gpu[idx_start + idx_sh +1] = Q_y;
                break;
          }else{
              K_x = Q_x + 1;
              K_y = Q_y - 1;
          }
      }else{
        P_x = Q_x - 1;
        P_y = Q_y + 1;
      }
    }
    x_start = X_gpu[idx_start + Z];
    y_start = Y_gpu[idx_start + Z];
    idx_start += Z;
  }
}

//######################### KERNEL to MERGE  ##################################
__global__ void mergeBig_k(int *X_gpu, int *Y_gpu, int *A_gpu, int *B_gpu, int *M_gpu, int dim_A, int dim_B){

  int idx_start = min(blockIdx.x * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int idx_end = min((blockIdx.x+1) * (dim_A+dim_B) / NSM, dim_A + dim_B);
  int x_start = X_gpu[idx_start];
  int y_start = Y_gpu[idx_start];

  __shared__ int X_sh[Z], Y_sh[Z];
  int idx_sh = threadIdx.x;

 while(idx_start + idx_sh < idx_end && idx_sh < Z){
    X_sh[idx_sh] = X_gpu[idx_start + idx_sh + 1];
    Y_sh[idx_sh] = Y_gpu[idx_start + idx_sh + 1];
    if(idx_sh == 0 && X_sh[0] > x_start)
      M_gpu[idx_start] = B_gpu[x_start];
    else if(idx_sh ==0)
      M_gpu[idx_start] = A_gpu[y_start];

    if( idx_sh>0 && X_sh[idx_sh]>X_sh[idx_sh-1])
      M_gpu[idx_start + idx_sh] = B_gpu[X_sh[idx_sh -1]];
    else if(idx_sh > 0)
      M_gpu[idx_start + idx_sh] = A_gpu[Y_sh[idx_sh -1]];

    x_start = X_sh[Z-1];
    y_start = Y_sh[Z-1];
    idx_start += Z;
  }
}
//########################### WRAPPER FIND PATH ###################################
void wrapper_FindPartition(int *a, int dim_A, int *b, int dim_B, int *m){
  int *A_GPU, *B_GPU, *M_GPU, *X_GPU, *Y_GPU;

  testCUDA(cudaMalloc(&A_GPU, dim_A*sizeof(int)));
  testCUDA(cudaMalloc(&B_GPU, dim_B*sizeof(int)));
  testCUDA(cudaMalloc(&X_GPU, (dim_A+dim_B+1)*sizeof(int)));
  testCUDA(cudaMalloc(&Y_GPU, (dim_A+dim_B+1)*sizeof(int)));
  testCUDA(cudaMalloc(&M_GPU, (dim_A+dim_B)*sizeof(int)));

  testCUDA(cudaMemcpy(A_GPU, a, dim_A*sizeof(int), cudaMemcpyHostToDevice));
  testCUDA(cudaMemcpy(B_GPU, b, dim_B*sizeof(int), cudaMemcpyHostToDevice));

  MergePath_partition<<<NSM,1>>>(A_GPU, dim_A, B_GPU, dim_B, X_GPU, Y_GPU);

  pathBig_k<<<NSM,Z>>>(X_GPU, Y_GPU, A_GPU, B_GPU, dim_A, dim_B);

  mergeBig_k<<<NSM,Z>>>(X_GPU, Y_GPU, A_GPU, B_GPU, M_GPU, dim_A, dim_B);

  testCUDA(cudaMemcpy(m, M_GPU, (dim_A+dim_B)*sizeof(int), cudaMemcpyDeviceToHost));

  testCUDA(cudaFree(A_GPU));
  testCUDA(cudaFree(B_GPU));
  testCUDA(cudaFree(M_GPU));
  testCUDA(cudaFree(X_GPU));
  testCUDA(cudaFree(Y_GPU));
}

//####################### PARALLEL MERGE SORT ############################
int * merge_parallel (int *T , int p, int q, int r){
  //A: fisrt array, B: second array, M_suport: mergesorted array
  int *A, *B, *M_support;
  int dim_A = q-p+1;
  int dim_B = r-q;
  if(dim_B > dim_A)
    std::swap(dim_A,dim_B);
  A = (int*)malloc(dim_A*sizeof(int));
  B = (int*)malloc(dim_B*sizeof(int));
  for(int i=0, j=p; i<q-p+1; ++i, ++j) //I copy in A the first array
    A[i] = T[j];
  for(int i=0, j=q+1; i<r-q; ++i, ++j) //I copy in B the second array
    B[i] = T[j];
  M_support = (int*)malloc((dim_A+dim_B)*sizeof(int));

//###### WRAPPER CALL #######
  wrapper_FindPartition(A, dim_A, B, dim_B, M_support);

  for(int i=p, j=0; i<=r; ++i, ++j) //I copy the sorted array into the original one
    T[i]=M_support[j];

  free(A);
  free(B);
  free(M_support);
  return T;
}

int * mergesort_parallel(int* T,int p,int r){
  if(p<r){ //It stops when p == r -> if(p!=r) -> it breaks in "SINGLETON"
//if(r-p<1e6)
    int q = floor((p+r)/2); //It divides the block {p,p+1,..,r} into two sub-blocks: {p,..,q} e {q+1,..,r}
    //I sort recursively the two sub-blocks
    //until I get two singletons (p==r): I SORT the SINGLETONS and then I FUSION them
    mergesort_parallel(T,p,q);
    mergesort_parallel(T,q+1,r);
    merge_parallel(T,p,q,r);
  }
  return T;
}

void parallel_sort(int* T, int N){
  mergesort_parallel(T,0,N-1);
}


// #############################################################################
// ################################ HACKAGE APP ################################
// #############################################################################


int EXPONENT = 65537;
int MODULUS = 13508703;

typedef int chaine;


// ####################### EXPONENTIATION SIMPLE ###############################

chaine pow(chaine a, int e)
{
    chaine x = a;
    chaine x_r = 1;
 
    while(e != 1)
    {
        if(e % 2)
          x_r = x_r*x;
        e = e/2;
        x = x*x;
    }

    return x*x_r;
}

// ####################### EXPONENTIATION MODULAIRE ############################

chaine mod_pow(chaine a, int e, chaine mod)
{
    chaine x = a;
    chaine x_r = 1;
 
    while(e != 1)
    {
        if(e % 2)
          x_r = x_r*x % mod;
        e = e/2;
        x = x*x % mod;

    }

    return x*x_r %mod;
}

// ####################### HASHER  #############################################

chaine hasher(int i, const char* name, int len_name)
{
    chaine x = i;
    for(int j = 0; j < len_name; j++)
    {
        x = x + x * (chaine)(name[j]) % MODULUS;
    }

    return mod_pow(x, EXPONENT, MODULUS);
}

void every_possibilities(chaine *vect, const char* name, int len_name, int NB_POSS)
{
    for(int i = 0; i < NB_POSS; i++)
    {
        vect[i] = hasher(i + 13, name, len_name);
    }
    
}

// ####################### TEST REMPLISSAGE ####################################
void test_every_poss(int NB_POSS)
{
    chaine v[NB_POSS];
    every_possibilities(v, "bob", 3, NB_POSS);
  
    for(int i = 0; i < NB_POSS; i++)
    {
        printf("%d ", v[i] );
    }
    printf("\n");
 
    printf("%d\n", mod_pow(32, 27, 37));
}

// ####################### CONDITIONS ON THE CHAIN  ############################

__global__ void find_the_good_chain(chaine *V, chaine *rep, int taille){
  // Fill the response if it founds good chains
  int idx = threadIdx.x;

  if(idx == 0 or idx >= taille)
    return;
  if((V[idx] & 0xFFFF000) ==  (V[idx-1] & 0xFFFF000))
  {
    rep[0] = V[idx];
    rep[1] = V[idx-1];
  }
}

// ################################ MAIN #######################################
int main()
{
// ----------------- Initialize data ---------------------------
    printf("Initialize and sort every possible vectors ... ");
 
    // Generate every possibilities for Alice and Bob
    int NB_POSS = 40000;
    chaine valice[NB_POSS];
    every_possibilities(valice, "alice", 5, NB_POSS);
 
    chaine vbob[NB_POSS];
    every_possibilities(vbob, "bob", 3, NB_POSS);

    chaine vres[2*NB_POSS];
 
    // Sort vectors
    std::sort(vbob, vbob+NB_POSS);
    std::sort(vbob, vbob+NB_POSS);
    printf("Ok.\n\n-------------------\n\n");
// -------------------- start Timer -----------------------------
    printf("Alice and bob arrive ! Merge sort Alice's and Bob's vectors\n");
    //Initializating the timer:
    float TimerV;
    cudaEvent_t start, stop;
    testCUDA(cudaEventCreate(&start));
    testCUDA(cudaEventCreate(&stop));

// ----------------- Merge sort 2 vectors ----------------------
    testCUDA(cudaEventRecord(start,0));
    wrapper_FindPartition(valice, NB_POSS, vbob, NB_POSS, vres);

// ------------------- End timer -------------------------------
    testCUDA(cudaEventRecord(stop,0));
    testCUDA(cudaEventSynchronize(stop));
    testCUDA(cudaEventElapsedTime(&TimerV, start, stop));
    print_array(vres, 2*NB_POSS);
    printf("End. Execution time: %f ms\n \n", TimerV);
    testCUDA(cudaEventDestroy(start));
    testCUDA(cudaEventDestroy(stop));
 
// ----------------- Get the appropriate chain -----------------
    printf("-------------------n\nTry to find 2 chains with the same first bits\n");
    int * V_GPU, * answer, *answer_GPU;
    answer = (chaine*)malloc(2*sizeof(chaine));
    answer[0] = -13;
    answer[1] = -13;
 
    testCUDA(cudaMalloc(&V_GPU, 2*NB_POSS*sizeof(chaine)));
    testCUDA(cudaMalloc(&answer_GPU, 2*sizeof(chaine)));
    testCUDA(cudaMemcpy(V_GPU, vres, 2*NB_POSS*sizeof(int), cudaMemcpyHostToDevice));
    testCUDA(cudaMemcpy(answer_GPU, answer, 2*sizeof(int), cudaMemcpyHostToDevice));
 
    printf("Begining of the search... ");
    find_the_good_chain<<<1,1024>>>(V_GPU, answer_GPU, 2*NB_POSS);
    printf("End of the search.\n");
 
    testCUDA(cudaMemcpy(answer, answer_GPU, 2*sizeof(chaine), cudaMemcpyDeviceToHost));
 
    testCUDA(cudaFree(V_GPU));
    testCUDA(cudaFree(answer_GPU));
  
    printf("2 keys are:\n");
    std::cout << std::hex << answer[0] << '\n';
    std::cout << std::hex << answer[1] << '\n';
    //printf("%d\n%d\n", answer[0] & 0xFFF000, answer[1] & 0xFFF000);
}

'File written in /content/src/main.cu'

In [ ]:
!cd src && make

nvcc -lineinfo main.cu -o main  
main.cu(22): warning: variable "P_x" was set but never used

main.cu(70): warning: variable "P_x" was set but never used



In [ ]:
!cd src && make run

./main
Initialize and sort every possible vectors ... Ok.

-------------------

Alice and bob arrive ! Merge sort Alice's and Bob's vectors
dim: 80000 
-13507636 -13507255 -13506814 -13506804 -13506753 -13506653 -13506424 -13506038 -13504435 -13502938  ... 13498008 13498407 13498528 13498888 13500677 13502455 13503343 13503804 13504497 13506701 
End. Execution time: 0.760576 ms
 
-------------------n
Try to find 2 chains with the same first bits
Begining of the search... End of the search.
2 keys are:
ff36e7fe
ff36e707
